In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import bokeh
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, output_file, show
from bokeh.models import Legend
from bokeh.models import LegendItem
from bokeh.models import HoverTool
from bokeh.transform import dodge
from bokeh.models import ColorBar, ColumnDataSource
from bokeh.models import  CustomJSHover
from bokeh.palettes import Spectral6
from bokeh.plotting import figure, output_file, show
from bokeh.transform import linear_cmap
from bokeh.models import HoverTool
from bokeh.layouts import layout
from bokeh.models.widgets import Tabs, Panel
from bokeh.io import curdoc
from bokeh.io import output_notebook
from bokeh.plotting import figure
from bokeh.util.warnings import BokehUserWarning 
from bokeh.resources import CDN
from bokeh.embed import file_html

from scipy import stats
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from dtreeviz.trees import dtreeviz
from scipy.interpolate import make_interp_spline
from sklearn.metrics import ConfusionMatrixDisplay

import json
import geopandas as gpd;
import datetime

from shapely.geometry import MultiPolygon
from shapely.geometry import shape
from shapely.geometry.polygon import Polygon

import branca
from branca.colormap import linear
import branca.colormap as cm

import folium;
from folium.plugins import StripePattern;
from folium.map import FeatureGroup
from folium.plugins import TimeSliderChoropleth
from folium.plugins import HeatMap

import json
import geopandas as gpd;
import datetime

from shapely.geometry import MultiPolygon
from shapely.geometry import shape
from shapely.geometry.polygon import Polygon

from branca.colormap import linear
import branca.colormap as cm
from geopy.distance import great_circle

In [ ]:
output_notebook()

# Motivation of the project
Smoking is NOT cool at all: it remains the leading cause of preventable disease, disability, and death in the United States. Youth e-cigarette use arises new public health issues in recent years, which is report that almost 85 percent of the more than 2 million middle and high school students have used e-cigarettes in USA in 2021. This topic soon rewind our memories back to high school time, when we had a friend with his cigarette all the time since then. He carried a four-inch long air-polluter and thinking themselves as the coolest people all over the world in the chocking smoke. For times we were trying to talk him into quitting, but he would always come back to enjoy his little e-cigarette in cola flavor again and again (Check the video below! He’s there!).

Good health and well-being is one of the primary topics in sustainable development. Encouraging smoking would be an enormous impedance in terms of achieving the sustainable goals, due to tobacco’s science-proved damage to human health. Inspired by our friend, we are wondering how many high schoolers out there are just like him. We would aldo like to dig further on studying the factors contributing to their addiction to cigarette via data-driven methods.

# Datasets
Tobacco control is always a heated topic within sustainability, especially the smoking behaviors among teeangers. It is part of the UN's sustainable goal to regulate the average age of tobacco users, and to avoid teenagers to smoke with full care and support.

In this project we are going to analyze NYC high school students' smoking behaviors, and to be more specific, how old are they do they start smoking. To dive deep on this issue, we combined three open dataset: the USA Youth Risk Behavior Surveillance Survey results, the NYC School Survey results, the 2019 NYC high school directory, and 2019 NYC licensed tobacco retailers. 

The first two are survey results gathered from students, where YRBSS dataset contains in total 500,000 high school students' individual responses every other year from 1991 to 2019 all across USA, and NYC School Survey gives statistics of over 1000 schools of 5 boroughs in NYC every year aiming to evaluate the school's quality. With NYC high school directory we can focus on the high schools students only, and tobbaco retailers help to further expand our vision to see how commercialization of tobacco relates to exact smoking behaviors of teenagers.

To operate on a sufficient dataset, we extracted YRBSS results from 2007, because NYC School Survey started from 2007, and the NYC school survey in 2017 to 2019 because questions and answers have been consistent within these three years.

The data cleaning and preprocessing procedures are scattered through the following sections, including but not limited to filter needed columns, encoding boroughs and binarize values. Please refer to the specific section to see what we have done on these datasets and some basic stats.

# YRBSS
## 1. Read the dataframe and data overview 

-  Read all the YRBSS results of the US dated from 1991
-  Extract data of 5 boroughs of New York from 2007, because NYC school survey starts from 2007, make it possible for further comprehensive study.
-  Show the overview of the dataset for all the following analysis in this notebook: total each year, total number of each borough in all years and each year.


 


In [ ]:
### read the full dataset with the chosen columns
cols = ['sitecode', 'sitename', 'sitetype', 'sitetypenum', 'year', 'stratum', 'PSU', 'record', 'age', 'sex', 'grade', 
        'race4', 'race7', 'stheight', 'stweight', 'bmi', 'bmipct', 'qnobese', 'qnowt', 'q66', 'q65', 
        'sexid', 'sexid2', 'sexpart', 'sexpart2', 'q8', 'q9', 'q10', 'q11', 'q12', 'q13', 'q14',
        'q15', 'q16', 'q17', 'q18', 'q19', 'q20', 'q21', 'q22', 'q23', 'q24', 'q25', 'q26', 'q27', 
        'q28', 'q29', 'q30', 'q31', 'q32', 'q33', 'q34', 'q35', 'q36', 'q37', 'q38', 'q39', 'q40', 
        'q41', 'q42', 'q43', 'q44', 'q45', 'q46', 'q47', 'q48', 'q49', 'q50', 'q51', 'q52', 'q53', 
        'q54', 'q55', 'q56', 'q57', 'q58', 'q59', 'q60', 'q61', 'q62', 'q63', 'q64', 'q67', 'q68', 
        'q69', 'q70', 'q71', 'q72', 'q73', 'q74', 'q75', 'q76', 'q77', 'q78', 'q79', 'q80', 'q81', 
        'q82', 'q83', 'q84', 'q85', 'q86', 'q87', 'q88', 'q89']

df_allyears = pd.read_table("./SADCQ.csv", sep=',', header='infer',usecols = cols)

### print the sitecode, and find the 5 boroughs of NYC
df_allyears['sitecode'].unique()

In [ ]:
### extract data of 5 boroughs after 2007
boroughs_code = ['NYG', 'NYH', 'NYI', 'NYJ', 'NYK']
boroughs_name = ['Bronx','Brooklyn','Manhattan','Queens','Staten Island']
df_boroughs = dict()

for i,b in enumerate(boroughs_code):
    df_boroughs[boroughs_name[i]] = df_allyears[(df_allyears['sitecode']==b) & (df_allyears['year']>=2007)]
    
# df_Bronx = df_boroughs['NYG']
# df_Brooklyn = df_boroughs['NYH']
# df_Manhattan = df_boroughs['NYI']
# df_Queens = df_boroughs['NYJ']
# df_Staten = df_boroughs['NYK']

### construct years list
years = df_boroughs['Bronx'].year.unique()

In [ ]:
### 
year_count = pd.DataFrame(index=list(years)+['total'])

for i in range(5):
    b = df_boroughs[boroughs_name[i]]
    b_years = b.groupby('year').count()['record']
    b_years = b_years.to_frame()
    b_years.rename(columns={"record":boroughs_name[i]},inplace=True)
    year_count = pd.concat((year_count, b_years), join='outer', axis=1) 
    
    year_count.iloc[7,i] = year_count.iloc[:,i][:-1].values.sum()
year_count['year_total'] = year_count.iloc[:,:-1].sum(axis=1)
    
#year_count = year_count.T
#year_count.reset_index(inplace=True)
#year_count

### 
borough_count = year_count.loc['total',:].to_frame()
# borough_count

###
year_proportion = pd.DataFrame()
for i,y in enumerate(years):
    year_proportion[y] = year_count.iloc[i,:-1]/year_count.loc[y,'year_total']

In [ ]:
color_list = ['#3A543F', '#ee8cb3', '#f6d4c8', '#71b9b6',
               '#017c8c', '#f1b638', '#07606a', '#c7f0d4', '#C2C2C2']

#### Fig1

In [ ]:
plt.figure(figsize=(8,4))

year_count['year_total'].drop('total').plot(color=color_list[4])
plt.xticks(year_count.index[:-1], year_count.index[:-1])
plt.title('YRBSS total count of NYC')
plt.ylabel('number of records')

plt.savefig('../fig/part_1_1.svg', bbox_inches='tight')

#### Fig2

In [ ]:
plt.figure(figsize=(8,4))

for i in range(5): 
    plt.bar(i, year_count[boroughs_name[i]].total, width=0.6, color=color_list[i+1])  
plt.xticks(range(5), boroughs_name)
plt.title("YRBSS total count of NYC's 5 boroughs 2007-2019")
plt.ylabel('number of records')

plt.savefig('../fig/part_1_2.svg', bbox_inches='tight')

#### Fig3

In [ ]:
plt.figure(figsize=(12,4))

x = np.linspace(0,30,7)-0.7

for i in range(5):    
    # plt.bar(x[i]+0.7, year_count[boroughs_name[i]].loc['total'], color = color_list[i+1], width=3)
    plt.bar(x + 0.7*i, year_count[boroughs_name[i]].drop('total'), color = color_list[i+1], width=0.6, label= boroughs_name[i])


plt.xticks(np.linspace(0,30,7)+0.7, year_count.index[:-1])
plt.ylabel('number of records')
plt.title('YRBSS yearly count of the 5 boroughs')
plt.legend(bbox_to_anchor=(1,1.03))

plt.savefig('../fig/part_1_3.svg', bbox_inches='tight')

#### Fig4

In [ ]:
year_proportion.T.plot(figsize=(12,4),color=color_list[1:])
plt.xticks(year_count.index[:-1], year_count.index[:-1])
plt.title('YRBSS yearly proportion of the 5 boroughs')
plt.ylabel('proportion')
plt.legend(bbox_to_anchor=(1,1.03))

plt.savefig('../fig/part_1_4.svg', bbox_inches='tight')

#### Comment
- The YRBSS dataset have different number of records each year. 2007, 2013 and 2015 have smaller amount at about 7500, and the rest years each have about 8000-10000 records.
- The YRBSS dataset records quite balanced amounts of the 5 boroughs. Bronx, Brooklyn and Manhattan each have about 15000 records in total, and Queens and Staten Island each have about 12000.
- The 5 boroughs are not equally sampled through the years. The numbers of records in each borough each year show no steady pattern, although the yearly proportion proves that the boroughs are sampled with similar weights through years, which usually Bronx, Brooklyn and Manhatten are sampled more.

## 2. Demographic overview

### 2.1 gender

In [ ]:
width = 600
height = 400

In [ ]:
# gender distribution
gender_ratio = pd.DataFrame(index=['female','male'])

for i in range(5):
    b = df_boroughs[boroughs_name[i]]
    b_gender = b.groupby('sex').count()['record']
    gender_ratio[boroughs_name[i]] = b_gender.values/b_gender.sum()
gender_ratio    

In [ ]:
### edit the ratio dataframe for bokeh plotting
types = ['female', 'male']

# gender_ratio.index = types
gender_ratio = gender_ratio.T
gender_ratio['borough'] = gender_ratio.index
gender_ratio['top']=1


### read into bokeh source and plot with stacked bar chart      
source = ColumnDataSource(data=gender_ratio);

p_gender = figure(title="Gender distribution of each borough",
           width=width, height=height,
           x_range = boroughs_name,
           x_axis_label='Borough', y_axis_label='Proportion')

legend_list = []
for idx,t in enumerate(types):
    if t=='female':
        bar = p_gender.vbar(x="borough", top=types[idx], source=source, width=0.5, fill_alpha=0.8, line_alpha=0,
         fill_color=color_list[1])
        legend_list.append(LegendItem(label=t, renderers=[bar]))
        
    if t=='male':
        bar = p_gender.vbar(x="borough", bottom=types[idx-1], top="top", source=source, width=0.5, fill_alpha=0.8, line_alpha=0,
         fill_color=color_list[0])
        legend_list.append(LegendItem(label=t, renderers=[bar]))

        

    if t =='female':
        hover = HoverTool(tooltips=[
         ('% of female', '@female{:.2%}')], 
         renderers=[bar])
        p_gender.add_tools(hover)
        
    if t =='male':
        hover = HoverTool(tooltips=[
         ('% of age male', '@male{:.2%}')], 
         renderers=[bar])
        p_gender.add_tools(hover)
        
        
p_gender.add_layout(Legend(items=legend_list, location=(10, 145)), 'right')
show(p_gender)

### 2.2 race

#### check race7 distribution
The dataset provides two records of racial information: 7-class `race7`  and 4-class `race4`, in which `race7` classifies further on minor subgroups of `race4`. First we had a look on `race7` to see if it's necessary to use 7 groups of race, and the result shows that the 4-group races are sufficient, because the minor groups of `race7` are really really small in amount.

(See plot below `American Indian/Alaska Native','Asian`, `Native Hawaiian/Other Pacific Islander`,`Multiple Races(Non-Hispanic)`)

In [ ]:
# race distribution
race7 = ['American Indian/Alaska Native','Asian','Black/African American','Hispanic/Latino',
         'Native Hawaiian/Other Pacific Islander', 'White','Multiple Races(Non-Hispanic)']

race_count = pd.DataFrame(index=race7)

for i in range(5):
    b = df_boroughs[boroughs_name[i]]
    b_race7 = b.groupby('race7').count()['record']
    race_count[boroughs_name[i]] = b_race7.values
# race_count

In [ ]:
race_count.plot(kind='line',rot=90, color=color_list[1:])

#### plot race4

In [ ]:
race4 = ['White','BlackAfricanAmerican','HispanicLatino','Other']
race_count = pd.DataFrame(index=race4)

for i in range(5):
    b = df_boroughs[boroughs_name[i]]
    b_race4 = b.groupby('race4').count()['record']
    race_count[boroughs_name[i]] = b_race4.values/b_race4.sum()
    
race_count    

In [ ]:
### edit the ratio dataframe for bokeh plotting

types = race4

race_count = race_count.T
race_count.columns = race4
race_count['borough'] = race_count.index

race_count
race_count['White_bottom'] = 0
race_count['White_top'] = race_count['White']


for i in range(len(types)):
    if i>=1:
        race_count[types[i]+'_bottom'] = race_count[types[i-1]+'_top']
        race_count[types[i]+'_top'] = race_count[types[i]+'_bottom'] + race_count[types[i]]
race_count

### read into bokeh source and plot with stacked bar chart      
source = ColumnDataSource(data=race_count);

p_race = figure(title="Race distribution of each borough",
           width=width, height=height,
           x_range = boroughs_name,
           x_axis_label='Borough', y_axis_label='Proportion')

legend_list = []
for idx,t in enumerate(types):
    bar = p_race.vbar(x="borough",  bottom = t+'_bottom', top=t+'_top',
                        source=source, width=0.5, fill_alpha=0.8, line_alpha=0,
                        fill_color=color_list[idx])
    
    legend_list.append(LegendItem(label=t, renderers=[bar]))
        

        

    if t =='White':
        hover = HoverTool(tooltips=[
         ('% of White', '@White{:.2%}')], 
         renderers=[bar])
        p_race.add_tools(hover)
        
    if t =='BlackAfricanAmerican':
        hover = HoverTool(tooltips=[
         ('% of BlackAfricanAmerican', '@BlackAfricanAmerican{:.2%}')], 
         renderers=[bar])
        p_race.add_tools(hover)
        
    if t =='HispanicLatino':
        hover = HoverTool(tooltips=[
         ('% of HispanicLatino', '@HispanicLatino{:.2%}')], 
         renderers=[bar])
        p_race.add_tools(hover)
        
    if t =='Other':
        hover = HoverTool(tooltips=[
         ('% of Other', '@Other{:.2%}')], 
         renderers=[bar])
        p_race.add_tools(hover)
        
p_race.add_layout(Legend(items=legend_list, location=(10, 145)), 'right')
show(p_race)

#### - Fig5 race_year

In [ ]:
race_year = pd.DataFrame()
for i in range(5):
    b = df_boroughs[boroughs_name[i]]
    race_year[boroughs_name[i]] = b.groupby(by=['race4','year']).count()['record']/np.array([year_count[boroughs_name[i]].values[:-1]]*4).flatten()

race_year['race'] = [i[0] for i in race_year.index.tolist()]
race_year['year'] = [i[1] for i in race_year.index.tolist()]

fig, axs = plt.subplots(2,3, figsize = ([16,10]), sharey=True, squeeze=False)
fig.suptitle('Race distribution of all boroughs through years')
plt.subplots_adjust(wspace=0.1, hspace=0.4)

for i in range(5):
    sns.lineplot(ax = axs[int(i/3),int(i%3)], x=race_year.year, y=race_year[boroughs_name[i]], 
                 palette='Set2', hue=race_year.race, legend=False)
    sns.scatterplot(ax = axs[int(i/3),int(i%3)], x=race_year.year, y=race_year[boroughs_name[i]], 
                    palette='Set2', hue=race_year.race, legend=False)
    axs[int(i/3),int(i%3)].title.set_text(boroughs_name[i])
    axs[int(i/3),int(i%3)].set_ylabel('Proportion')

fig.legend(race4,bbox_to_anchor=(0.68, 0.95),ncol=4)
fig.delaxes(axs[1][2])
plt.savefig('../fig/part_1_5_race_year.svg', bbox_inches='tight')

In [ ]:
# race_year

### 2.3 age/grade
`age` and `grade` both reveals information about age, so we exmined in detail of these, and see if they are strongly correlated.

#### check age distribution
Basically the age samples across boroughs are repeating a same pattern, (see in the grouped bar chart below). We created an interactive plot to help readers know this more clearly.

In [ ]:
age = [12,13,14,15,16,17,18]
grade =[9,10,11,12]

age_count = pd.DataFrame(index=age)

for i in range(5):
    b = df_boroughs[boroughs_name[i]]
    b_age = b.groupby('age').count()['record']
    age_count[boroughs_name[i]] = b_age.values
age_count

In [ ]:
age_count.T.plot(color = color_list,kind='bar', legend=False)
plt.legend(bbox_to_anchor=(1.1, 1.2),ncol=7)

In [ ]:
age_ratio=pd.DataFrame(index=age)
for i in range(5):
    b = df_boroughs[boroughs_name[i]]
    b_age = b.groupby('age').count()['record']

    age_ratio[boroughs_name[i]] = b_age.values/b_age.sum()
age_ratio

In [ ]:
### edit the ratio dataframe for bokeh plotting
types = [str(a) for a in age]

age_ratio.index = types
age_ratio = age_ratio.T
age_ratio['borough'] = age_ratio.index
age_ratio

age_ratio['12_bottom'] = 0
age_ratio['12_top'] = age_ratio['12']


for i in range(len(types)):
    if i>=1:
        age_ratio[types[i]+'_bottom'] = age_ratio[types[i-1]+'_top']
        age_ratio[types[i]+'_top'] = age_ratio[types[i]+'_bottom'] + age_ratio[types[i]]
        

### read into bokeh source and plot with stacked bar chart      
source = ColumnDataSource(data=age_ratio);

p_age = figure(title="Age distribution of each borough",
           width=width, height=height,
           x_range = boroughs_name,
           x_axis_label='Borough', y_axis_label='Proportion')

legend_list = []
for idx,t in enumerate(types):
        
    bar = p_age.vbar(x="borough", bottom = t+'_bottom', top=t+'_top', source=source, width=0.5, fill_alpha=0.8, line_alpha=0,
         fill_color=color_list[idx])
    
    if t =='12':
        legend_list.append(LegendItem(label='<=12', renderers=[bar]))
    elif t =='18':
        legend_list.append(LegendItem(label='>=18', renderers=[bar]))
        
    else:
        legend_list.append(LegendItem(label=t, renderers=[bar]))
        

    if t =='12':
        hover = HoverTool(tooltips=[
         ('% of age group <= 12', '@12{:.2%}')], 
         renderers=[bar])
        p_age.add_tools(hover)
        
    if t =='13':
        hover = HoverTool(tooltips=[
         ('% of age group '+t, '@13{:.2%}')], 
         renderers=[bar])
        p_age.add_tools(hover)
        
    if t =='14':
        hover = HoverTool(tooltips=[
         ('% of age group '+t, '@14{:.2%}')], 
         renderers=[bar])
        p_age.add_tools(hover)
        
    if t =='14':
        hover = HoverTool(tooltips=[
         ('% of age group '+t, '@14{:.2%}')], 
         renderers=[bar])
        p_age.add_tools(hover)
        
    if t =='15':
        hover = HoverTool(tooltips=[
         ('% of age group '+t, '@15{:.2%}')], 
         renderers=[bar])
        p_age.add_tools(hover)
    if t =='16':
        hover = HoverTool(tooltips=[
         ('% of age group '+t, '@16{:.2%}')], 
         renderers=[bar])
        p_age.add_tools(hover)
        
    if t =='17':
        hover = HoverTool(tooltips=[
         ('% of age group '+t, '@17{:.2%}')], 
         renderers=[bar])
        p_age.add_tools(hover)
        
    if t =='18':
        hover = HoverTool(tooltips=[
         ('% of age group >= 18', '@18{:.2%}')], 
         renderers=[bar])
        p_age.add_tools(hover)
        
p_age.add_layout(Legend(items=legend_list, location=(10, 145)), 'right')
show(p_age)

#### check grade distribution

In [ ]:
grade_count = pd.DataFrame(index=grade)

for i in range(5):
    b = df_boroughs[boroughs_name[i]]
    b_grade = b.groupby('grade').count()['record']
    grade_count[boroughs_name[i]] = b_grade.values
grade_count

#### - Fig6 grade

In [ ]:
grade_count.T.plot(kind='bar',color=color_list,figsize=(8,4), rot=0, ylabel='number of records',title='Grade count of each borough')
plt.legend(bbox_to_anchor=(1.15, 1.03), ncol=1)
plt.savefig('../fig/part_1_6_grade.svg', bbox_inches='tight')

#### check grade-age disctribution

In [ ]:
grade_age_count = pd.DataFrame()
for i in range(5):
    b = df_boroughs[boroughs_name[i]]
    #b_grade_age = b.groupby(['grade','age']).count()['record']
    grade_age_count[boroughs_name[i]]  = b.groupby(['grade','age']).count()['record']/borough_count.iloc[i].values
grade_age_count

#### - Fig7 grade-age

In [ ]:
fig, axs = plt.subplots(2,2, figsize = ([12,8]))
plt.suptitle('Age distribution in each grade')
plt.subplots_adjust(wspace=0.2, hspace=0.4)
for i in range(4):
    axs[int(i/2),int(i%2)].set_title('grade '+str(grade[i]))
    axs[int(i/2),int(i%2)].set_ylabel('proportion')
    axs[int(i/2),int(i%2)].set_xticks(range(1,8), ['<=12']+types[1:-1]+['>=18'])
    for j,b in enumerate(boroughs_name):
        sns.lineplot(x=grade_age_count.T[i+1].columns,y=grade_age_count.T[i+1].T[b], 
                     ax = axs[int(i/2),int(i%2)], color = color_list[j+1], label=b, legend=False)

fig.legend(boroughs_name, bbox_to_anchor=(1.05, 0.89), ncol=1)
plt.savefig('../fig/part_1_7_grade_age.svg', bbox_inches='tight')

In [ ]:
df_boroughs5 = pd.DataFrame()

for i in range(5):
    b = df_boroughs[boroughs_name[i]]
    df_boroughs5 = pd.concat((df_boroughs5,b))
    #b_grade_age = b.groupby(['grade','age']).count()['record']
    #grade_age_count[boroughs_name[i]]  = b.groupby(['grade','age']).count()['record']
    
#y=grade_age_count.T[1].T
#y

In [ ]:
fig, axs = plt.subplots(2,2,figsize = (12,8))

for g in range(4):
    for b in boroughs_code:
        sta = df_boroughs5[(df_boroughs5['sitecode']==b) & (df_boroughs5['grade']==g+1)].age.dropna().values
        sns.kdeplot(sta, ax= axs[int(g/2),int(g%2)], shade=None)
    

#ax.set_xticklabels(focuscrimes, rotation = 65)

#### - Fig8 tab stacked bar

In [ ]:
l1 = layout([[p_age]], sizing_mode='fixed')
l2 = layout([[p_gender]], sizing_mode='fixed')
l3 = layout([[p_race]], sizing_mode='fixed')

tab1 = Panel(child=l1, title="Age")
tab2 = Panel(child=l2, title="Gender")
tab3 = Panel(child=l3, title="Race")
tabs = Tabs(tabs=[tab2, tab3, tab1])

curdoc().add_root(tabs)

html = file_html(tabs, CDN, "my plot")
file = open("../fig/part_1_demographic.html","w")
file.write(html)
file.close()
### individual plot cell to avoid re-assigning problem
show(tabs)

### Comments of demographic overview

#### Gender
The dataset contains fairly unbiased distribution on genders, with gender ratio approximate to 1:1 for each borough, except that Manhattan shows a slightly higher proportion of female respondants with about 60%.

#### Race
- Respondants of the 5 boroughs don't share the same race distribution. Bronx, Manhatten and Queens are dominated by Hispanic/Latino students, while Brooklyn reports more Black/African American participants. To our surprise, Staten Island seems to be the only borough that are mostly comprised of White students.
- Seeing from the line chart of the four ethnic groups in each borough, the race distribution of each borough remains rather steady through the years. Only few intersections, such as Black/African American with Hispanic/Latino in Brooklyn and White with Hispanic/Latino in Staten Island are shown, implying small fluctuations of the racial diversity.

#### Age/Grade
- The dataset is sampled in a similar distribution of age groups across boroughs, where 14-17 year-old participants takes part in the majority. Age 14, 15, 16 and 17 each contributes roughly 20%-25% of the total samples in each borough.
- Similarly, the grade distribution is also the same across boroughs. The dataset is of high school students, so grade 9-12 are evenly sampled in all 5 boroughs.
- Further check on the age distribution by grade across all boroughs, it can be seen that grade 9 students are centered around 14 years old, grade 10 around 15 years old, and so on and so forth. There are no signs of early or late schooling in any borough, because their pattern of grade-age distribution is almost the same.

## 3. Smoking behavior analysis

In [ ]:
### record possible useful question lists
bullied = ['q23','q24']
mentality_suicide = ['q25','q26','q27','q28','q29']

cigarette = ['q30','q31','q32','q33']
cigarette_start = ['q31']
e_vapor = ['q34','q35','q36']
other_nicotine_product = ['q37','q38']
quit_tobacco = ['q39']

alcohol = ['q40','q41','q42','q43','q44']
drugs = ['q45','q46','q47','q48','q49','q50','q51','q52','q53','q54','q55','q56','q57']
exercise = ['q78','q81','q82']
TV_computer = ['q79','q80']
dentisit = ['q86']
sleep = ['q87']
grade = ['q89']

### 3.1 Some preprocessing: check if a question is in the survey each year

'q30','q32','q33': have records all years  
'q31': only valid in 2019

In [ ]:
for y in years:
    for i,b in enumerate(boroughs_name):
        sta_borough = df_boroughs[b]
        for question in cigarette:
            print(y,b,question, pd.notnull(sta_borough[sta_borough['year']==y][question]).sum())

### 3.2 Smoking cigarette in each borough  
In this section we analyze these four question:

q30 Have you ever tried cigarette?  
- 1.0: yes  
- 2.0: no  

q31 How old were you when you first tried smoking?
- 1.0: never tried
- 2.0: <=8 years old
- 3.0: 9 or 10 years old
- 4.0: 11 or 12 years old
- 5.0: 13 or 14 years old
- 6.0: 15 or 16 years old
- 7.0: >=17 years old

q32 How many days did you smoke in the last month?
- 1.0: 0 days 
- 2.0: 1 or 2 days
- 3.0: 3 to 5 days 
- 4.0:  6 to 9 days 
- 5.0:  10 to 19 days
- 6.0:  20 to 29 days 
- 7.0:  All 30 days 
 
q33 How many cigarettes did you smoke per day on the days you have smoked in the last month?
- 1.0: 0 
- 2.0: <1
- 3.0: 1 
- 4.0: 2 to 5 
- 5.0: 6 to 10
- 6.0: 11 to 20 
- 7.0: >20



In [ ]:
tried_cigarette = pd.DataFrame()
smoke_days_per_month = pd.DataFrame()
cigarette_per_day = pd.DataFrame()

for i,b in enumerate(boroughs_name):
    sta_borough = df_boroughs[b]
    tried_cigarette[b] = sta_borough.groupby('q30').count()['record']/len(sta_borough)
    smoke_days_per_month[b] = sta_borough.groupby('q32').count()['record']#/len(sta_borough)
    cigarette_per_day[b] = sta_borough.groupby('q33').count()['record']#/len(sta_borough)
tried_cigarette.index=['tried smoking','never tried smoking']
tried_cigarette

In [ ]:
geodata_path = "NYC.geojson";
with open(geodata_path) as f:
    geojson = json.load(f)

geojson_Queens = geojson['features'][0]['geometry'];
geojson_Brooklyn = geojson['features'][1]['geometry'];
geojson_Bronx = geojson['features'][4]['geometry'];
geojson_Manhattan = geojson['features'][3]['geometry'];
geojson_StatenIsland = geojson['features'][2]['geometry'];

geo_dict = {'Borough': ['Queens', 'Brooklyn', 'Bronx', 'Manhattan', 'StatenIsland'],
     'geometry': [shape(geojson_Queens), shape(geojson_Brooklyn), shape(geojson_Bronx), shape(geojson_Manhattan),
                  shape(geojson_StatenIsland)]};
gdf_borough = gpd.GeoDataFrame(geo_dict).set_index("Borough");

gdf_borough

#### - Fig9 - pct of smokers


In [ ]:
def geoplot_answer(mapp, df) :
    folium.TileLayer('Stamen Toner').add_to(mapp)
    is_first = True;

    for (col_name, col_data) in df.iteritems():
        if (col_name == "Borough") :
            continue;

        folium.Choropleth(
            geo_data=geojson,
            name="Answer distribution for option " + str(col_name),
            data=df,
            columns=["Borough", col_name],
            key_on="properties.boro_name",
            fill_color="BuPu",
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name="Answer rate for option " + str(col_name),
            show=is_first
        ).add_to(mapp)

        is_first = False;

    folium.features.GeoJson(name="Borough borders", data=geojson,
                        style_function=lambda x :{'color': "white", "weight": 2},
                        show=True).add_to(mapp)

    folium.LayerControl().add_to(mapp)

In [ ]:
cityhall_lat_lon = [40.7128, -74.0060];
mapp = folium.Map(location=cityhall_lat_lon,
                    tiles = "openstreetmap",
                    zoom_start = 11);

df = tried_cigarette.transpose().reset_index().rename(columns={"index":"Borough"});

geoplot_answer(mapp, df);
mapp.save("../fig/part_1_pct_smokers.html")
mapp

#### Comment
- Clearly, Staten Island and Queens reported more high school smokers - nearly 30% of high school students have tried smoking.
- Bronx, Brooklyn and Manhattan have a similar level of high school smokers, which is lower than the forementioned two boroughs, at about 22%.
#### ------------------------------------------------------------------------

In [ ]:
days_smoke = ['0 days', '1 or 2 days', '3 to 5 days', '6 to 9 days', '10 to 19 days', '20 to 29 days', 'All 30 days']
smoke_days_per_month.index = days_smoke
smoke_days_per_month

#### - Fig10.1

In [ ]:
### days that have smoked - pct among smokers
days_smoke = ['0 days', '1 or 2 days', '3 to 5 days', '6 to 9 days', '10 to 19 days', '20 to 29 days', 'All 30 days']
(smoke_days_per_month.iloc[1:,:]/smoke_days_per_month.iloc[1:,:].sum()*100).T.plot(figsize=(6,6), kind='bar', 
                                                                                   stacked=True, color=color_list, 
                                                                                   yticks= np.linspace(0,100,21),
                                                                                   rot=0)

plt.title('% of days of smoking last month'),
plt.ylabel('percentage(%)')
plt.legend(days_smoke[1:],bbox_to_anchor=(1,1.01))

plt.savefig('../fig/part_1_10_1_days_smoke_pct.svg', bbox_inches='tight')

In [ ]:
answers = smoke_days_per_month.iloc[1:,:]/smoke_days_per_month.iloc[1:,:].sum()*100

####  -  Fig10.2 - frequency of smoking of smokers

In [ ]:
cityhall_lat_lon = [40.7128, -74.0060];
mapp = folium.Map(location=cityhall_lat_lon,
                    tiles = "openstreetmap",
                    zoom_start = 11);

df = answers.transpose().reset_index().rename(columns={"index":"Borough"});

geoplot_answer(mapp, df);
mapp.save("../fig/part_1_days_smokers.html")
mapp

#### Comment
- Since the amount of non-smokers is still overwhelmingly larger than smokers in all boroughs, we only visualize the sub-categories of smokers and analyze characteristics of smokers' behaviors in each borough.
- Good news: In all boroughs, around 1/3 of the smokers only smoke occasionally, corresponding to the option 1 or 2 days in the last month. In Brooklyn and Manhatten the proportion is even larger, up to over 40%.
- Both Queens and Staten Island reported significantly more smokers who smoke on over 20 days in a month. In particular, Staten Island seems to be the hot zone of heavy smokers, with smallest percentage of those who only smoke 1 or 2 days and largest percentage of those who smoke on all 30 days. In fact the percentage (and absolute number, too) of day-to-day smokers in Staten Island is so large that it doubles that of Manhatten.
- Bronx shows a higher percentage of those who have smoked for 3 to 5 days. Be careful! These students are likely to more than just curious about cigarettes -- they are forming a habit of smoking!
#### ------------------------------------------------------------------------


In [ ]:
cigarette_day_smoke = ['0', '<1', '1', '2 to 5' ,'6 to 10', '11 to 20', '>20']
cigarette_per_day.index = cigarette_day_smoke
cigarette_per_day

In [ ]:
# (cigarette_per_day.iloc[1:,:]/cigarette_per_day.iloc[1:,:].sum()*100)

#### - Fig11.1

In [ ]:
(cigarette_per_day.iloc[1:,:]/cigarette_per_day.iloc[1:,:].sum()*100).T.plot(figsize=(6,6), kind='bar', 
                                                                                   stacked=True, color=color_list, 
                                                                                   yticks= np.linspace(0,100,21),
                                                                                  rot=0)
cigarette_day_smoke = ['0', '<1', '1', '2 to 5' ,'6 to 10', '11 to 20', '>20']
plt.title('% of cigarette per day last month'),
plt.ylabel('percentage(%)')
plt.legend(cigarette_day_smoke[1:],bbox_to_anchor=(1,1.01))

plt.savefig('../fig/part_1_11_1_per_day_pct.svg', bbox_inches='tight')

In [ ]:
answers = smoke_days_per_month.iloc[1:,:]/smoke_days_per_month.iloc[1:,:].sum()*100

#### - Fig11.2 -  number of cigarettes each day

In [ ]:
cityhall_lat_lon = [40.7128, -74.0060];
mapp = folium.Map(location=cityhall_lat_lon,
                    tiles = "openstreetmap",
                    zoom_start = 11);

df = answers.transpose().reset_index().rename(columns={"index":"Borough"});

geoplot_answer(mapp, df);
mapp.save("../fig/part_1_cigarettes_perday_smokers.html")
mapp

#### Comment
- Again on heavy smokers -- Stanten Island win! Over 10% participants smoked more than 10 cigarettes per day, while less than 50% consume up to 1 cigerette daily.
- Other 4 boroughs all have more than 50% participants smoking less than 1 cigerette, and more than 85% participants no more than 5 a day.
- Manhattan reports the largest percentage of students smoke no more than 1 cigerrete per day and the smallest percentage of smoking more than 5 cigerattes per day.
#### ------------------------------------------------------------------------

In [ ]:
#fig, axs = plt.subplots(2,3, figsize = ([16,10]), sharey=True, squeeze=False)
#plt.subplots_adjust(wspace=0.1, hspace=0.4)
#fig.delaxes(axs[1][2])

cigarette_initiation = pd.DataFrame()
cigarette_initiation_age = []
for i,b in enumerate(boroughs_name):
    sta_borough = df_boroughs[b]
    sta_borough = sta_borough[sta_borough['year']==2019]
    cigarette_initiation[b] = sta_borough.groupby('q31').count()['record']#/len(sta_borough)

    s = sta_borough[sta_borough['q31']!=1]['q31'].dropna()
    cigarette_initiation_age.append(s)
    #s.plot(kind='hist',ax=axs[int(i/3),int(i%3)], width=0.2)

In [ ]:
# cigarette_initiation.iloc[1:]/cigarette_initiation.iloc[1:].sum()

#### - Fig12  starting to smoke age distribution

In [ ]:
fig, axs = plt.subplots(1,2, figsize = ([14,6]))
fig.suptitle('Age distribution of starting smoking')
plt.subplots_adjust(wspace=0.3)
age_smoke = ['never tried','<=8 yo', '9/10 yo', '11/12 yo', 
             '13/14 yo', '15/16 yo', '>=17 yo']

(cigarette_initiation.iloc[1:]/cigarette_initiation.iloc[1:].sum()).T.plot(kind='bar', ax=axs[0], stacked=True, 
                                                                           color=color_list, 
                                                                           rot=0)
axs[0].set_title('% of age to start smoking')
axs[0].set_ylabel('percentage(%)')
axs[0].legend(age_smoke[1:],bbox_to_anchor=(0.86,-0.08),ncol=3)
axs[0].set_yticks(np.linspace(0,1,21),np.linspace(0,100,21))

box = dict(color='#1870c7')
whisker = dict(color='#1870c7')
cap = dict(color='#1870c7')

axs[1].boxplot(cigarette_initiation_age,boxprops=box,whiskerprops=whisker,capprops=cap)
axs[1].set_title('boxplot of starting smoking')
axs[1].set_ylabel('age')
axs[1].set_yticks(range(2,8),age_smoke[1:])
axs[1].set_xticks(range(1,6),boroughs_name)
#axs[1].legend(age_smoke,bbox_to_anchor=(-0.1,1.02))
plt.savefig('../fig/part_1_12_smoke_age_distr.svg', bbox_inches='tight')

#### Comment - NEED MORE
- Surprisingly, Bronx have significantly larger percentage of students who start smoking before 8 years old, which is about 30%. What's more, the distribution in Bronx is obviously asymmetrical, which leans expands more to younger groups.
- Manhattan has a great proportion of almost 40% of smokers picking up smoking at 13 or 14 years, and compared to other 4 boroughs, an overall older age of smoking acquisition.



### 3.3 Temporal change of tobacco usage

In [ ]:
yearly_smoke_percentage = pd.DataFrame(columns=boroughs_name,index=years)

for i,b in enumerate(boroughs_name):
    sta_borough = df_boroughs[b]
    for y in years:
        s = sta_borough[sta_borough['year']==y].groupby('q30').count()['record']
        yearly_smoke_percentage.loc[y,b] = s.values[0]/s.sum()
yearly_smoke_percentage

#### - Fig13.1 pct of smokers

In [ ]:
plt.figure(figsize=(8,4))
yearly_smoke_percentage.plot(kind='line',color=color_list[1:],figsize=(6,4))

plt.title('Yearly change of smokers proportion in students'),
plt.ylabel('percentage(%)')
plt.yticks(np.linspace(0,0.6,13),np.linspace(0,60,13))
plt.xticks(np.linspace(2007,2019,7))
plt.legend(bbox_to_anchor=(1.35,1.03))


plt.savefig('../fig/part_1_13_time_smokers.svg', bbox_inches='tight')

In [ ]:
yearly_smoke_percentage

In [ ]:
def geoplot_time_series(map, df, colors, title) :
    # folium.TileLayer('openstreetmap').add_to(map);
    # folium.TileLayer('Stamen Toner').add_to(map);

    # colorscale = branca.colormap.linear.YlGnBu_09.scale(0, 30);
    df = df.set_index("Borough");

    max_color = df.to_numpy().max();
    min_color = df.to_numpy().min();
    cmap = colors.scale(min_color, max_color);

    df = df.reset_index();

    df_color = df.copy()
    for (col_name, col_data) in df.iteritems():
            if (col_name == "Borough") :
                continue;

            df_color[col_name] = df[col_name].map(cmap)

    style_dict = {};

    for (_, row) in df_color.iterrows():
        inner_dict = {};

        for (col_name, col_data) in row.iteritems():
            if (col_name == "Borough") :
                continue;

            inner_dict[
                (int)((datetime.datetime(year=int(col_name), month=1,day=1)).timestamp())] = \
                                { 'color': row[int(col_name)], 'opacity': 0.7 };

        style_dict[row["Borough"].replace(' ', '')] = inner_dict;

    map.add_child(
        TimeSliderChoropleth(
            data=gdf_borough.to_json(), styledict=style_dict, overlay=True
        )
    );

    _ = cmap.add_to(map);
    cmap.caption = title;

#### - Fig13.2 - temporal (movie) in a map of smoking pct

In [ ]:
cityhall_lat_lon = [40.7128, -74.0060];
mapp = folium.Map(location=cityhall_lat_lon,
                    tiles = "openstreetmap",
                    zoom_start = 11);

df = yearly_smoke_percentage.transpose().reset_index().rename(columns={"index": "Borough"});

colors = branca.colormap.linear.YlGnBu_09;
title = "Yearly change of smoking rate"
geoplot_time_series(mapp, df, colors, title);
mapp.save("../fig/part_1_year_smoking_percentage.html")
mapp

#### Comment
- The high school students' smoking rate in all boroughs slide down quickly with time, starting from around 40%-50% in 2007 to around 10% in 2019. The slope remains steady for Bronx, Brooklyn and Manhatten, where the smoking rate decreses almost the same each year, while Queens were in a plateau in 2011-2015, and Staten Island went through a plummeting period in 2009-2011 and a remains stable after 2017.
- Queens and Staten Island have retained a high smoking rate through most years, and both landed on slightly higher smoking rate in 2019 than other 3 boroughs. Recall that these are also sufferers of heavy smokers and early smokers, it should really raise a flag.
#### ------------------------------------------------------------------------

In [ ]:
yearly_heavy_smokers_percentage = pd.DataFrame(columns=boroughs_name,index=years)

for i,b in enumerate(boroughs_name):
    sta_borough = df_boroughs[b]
    for y in years:
        s = sta_borough[sta_borough['year']==y].groupby('q33').count()['record']
        yearly_heavy_smokers_percentage.loc[y,b] = s.values[5:].sum()/s.values[1:].sum()
yearly_heavy_smokers_percentage

#### - Fig14.1 pct of smoke >10 cigarettes

In [ ]:
yearly_heavy_smokers_percentage.plot(color=color_list[1:])


plt.title('Yearly change of heavy smokers proportion in high school smokers'),
plt.ylabel('percentage(%)')

plt.xticks(np.linspace(2007,2019,7))
plt.legend(bbox_to_anchor=(1.35,1.03))

plt.savefig('../fig/part_1_14_heavy_smoker.svg', bbox_inches='tight')

In [ ]:
yearly_heavy_smokers_percentage

#### - Fig14.2

In [ ]:
cityhall_lat_lon = [40.7128, -74.0060];
mapp = folium.Map(location=cityhall_lat_lon,
                    tiles = "openstreetmap",
                    zoom_start = 11);

df = yearly_heavy_smokers_percentage.transpose().reset_index().rename(columns={"index": "Borough"});

colors = branca.colormap.linear.YlGnBu_09;
title = "Yearly change of rate of heavy smokers"
geoplot_time_series(mapp, df, colors, title);
mapp.save("../fig/part_1_year_heavy_smokers.html")
mapp

####  Comment
- The USA CDC have defined an indicator on top of q33 results, demonstrating the proportion of people who smoke more than 10 cigarettes per day among smokers. 
- There is a sharp decrese from 2017 to 2019 in Queens.
- Manhatten remains the lowest heavy smoker rate through the years.
- Heavy smoker rate fluctuates mildly in Brooklyn, with an average of about 10%.
- The heavy smoker rate in Staten Island and Bronx in general raises through the years. This phenomenon partly originated from the decrease in total number of smokers within these boroughs through the years.


### 3.4 Other tobacco products
**e-vapor**  
q34 Have you tried e-vapor product?
- 1.0: yes
- 2.0: no

q35 How many days did you use e-vapor product in the last month?
- 1.0:  0 days 
- 2.0:  1 or 2 days
- 3.0:  3 to 5 days 
- 4.0:  6 to 9 days 
- 5.0:  10 to 19 days
- 6.0:  20 to 29 days 
- 7.0:  All 30 days 
 
q36 During the past 30 days, how did you usually get e-vapor products?
- 1.0: I did not use e-vapor products
- 2.0: Store or gas station
- 3.0: On the Internet 
- 4.0: I paid someone else to buy them
- 5.0: Borrowed them from others
- 6.0: Legal person gave me
- 7.0: Took them from a store/family
- 8.0: Some other way 
 
**cigars and others**

q37 How many days did you use chewing tobacco, snuff, dip, snus or dissolvable tobacco products in the last month?
- 1.0:  0 days 
- 2.0:  1 or 2 days
- 3.0:  3 to 5 days 
- 4.0:  6 to 9 days 
- 5.0:  10 to 19 days
- 6.0:  20 to 29 days 
- 7.0:  All 30 days 

q38 How many days did you use cigars, cigarillos or little cigars in the last month?
- 1.0:  0 days 
- 2.0:  1 or 2 days
- 3.0:  3 to 5 days 
- 4.0:  6 to 9 days 
- 5.0:  10 to 19 days
- 6.0:  20 to 29 days 
- 7.0:  All 30 days 


#### check if the related questions are in the survey each year

In [ ]:
for y in years:
    for i,b in enumerate(boroughs_name):
        sta_borough = df_boroughs[b]
        for question in e_vapor:
            print(y,b,question, pd.notnull(sta_borough[sta_borough['year']==y][question]).sum())
        for question in other_nicotine_product:
            print(y,b,question, pd.notnull(sta_borough[sta_borough['year']==y][question]).sum())
        print(y,b,'q39', pd.notnull(sta_borough[sta_borough['year']==y]['q39']).sum())

'q34': 2019  
'q35': 2015, 2017, 2019  
'q36': 2019  
'q37': 2017  
'q38': no 2011  
'q39': no records

#### grab data on e-vapor products and cigars
q35 and q38 appear to have more abundant results to analyze, so we choose only these two questions.

In [ ]:
evapor_days_month = pd.DataFrame() #'q35'
cigars_days_month = pd.DataFrame() #'q38'

for i,b in enumerate(boroughs_name):
    sta_borough = df_boroughs[b]
    evapor_days_month[b] = sta_borough.groupby('q35').count()['record']#/sta_borough.groupby('q35').count()['record'].values[1:].sum()
    cigars_days_month[b] = sta_borough.groupby('q38').count()['record']#/sta_borough.groupby('q38').count()['record'].values[1:].sum()
    

#### explore evapor data

In [ ]:
evapor_no = (evapor_days_month.iloc[0]/evapor_days_month.sum())
evapor_no#.plot(kind='bar')

In [ ]:
evapor_days_percentage = evapor_days_month.iloc[1:]/evapor_days_month.iloc[1:].sum()
evapor_days_percentage

In [ ]:
evapor_days_percentage.T.plot(figsize=(6,6), kind='bar', stacked=True, color=color_list, rot=0)

plt.title('% of days of using e-vapor last month'),
plt.ylabel('percentage(%)')
plt.yticks(np.linspace(0,1,21),np.linspace(0,100,21))
plt.legend(days_smoke,bbox_to_anchor=(1,1.01))
plt.show()

#### explore cigars and alike

In [ ]:
cigars_days_month

In [ ]:
cigars_no = (cigars_days_month.iloc[0]/cigars_days_month.sum())
cigars_no#.plot(kind='bar')

In [ ]:
cigars_days_percentage = cigars_days_month.iloc[1:]/cigars_days_month.iloc[1:].sum()
cigars_days_percentage.T.plot(figsize=(6,6), kind='bar', stacked=True, color=color_list, rot=0)

plt.title('% of days of using e-cigars and alike last month'),
plt.ylabel('percentage(%)')
plt.yticks(np.linspace(0,1,21),np.linspace(0,100,21))
plt.legend(days_smoke,bbox_to_anchor=(1,1.01))
plt.show()

####  cigarettes correlated with evapor/cigars?

In [ ]:
df_boroughs5_from_2015 = df_boroughs5[(df_boroughs5['year']>=2015)][['q32','q35','q38']].dropna() 

for i in ['q32','q35','q38']:
    s = np.array(df_boroughs5_from_2015[i]-1,dtype=bool)
    df_boroughs5_from_2015[i+'_bin'] = np.array(s,dtype=int)
df_boroughs5_from_2015

#### ------------------------------------------------------------------------
#### - Fig15 comparing evapor and cigars

In [ ]:
fig, axs = plt.subplots(1,2, figsize = ([14,6]))
fig.suptitle('Age distribution of smoking e-vapor vs. cigars and alike')

plt.subplots_adjust(wspace=0.3)


evapor_days_percentage.T.plot(kind='bar', stacked=True, color=color_list, rot=0, ax=axs[0], legend=False)
axs[0].set_title('% of days of using e-vapor last month'),
axs[0].set_ylabel('percentage(%)')
axs[0].set_yticks(np.linspace(0,1,21),np.linspace(0,100,21))
# axs[0].legend(days_smoke,bbox_to_anchor=(1,1.01))


cigars_days_percentage = cigars_days_month.iloc[1:]/cigars_days_month.iloc[1:].sum()
cigars_days_percentage.T.plot(kind='bar', stacked=True, color=color_list, rot=0, ax=axs[1],legend=False)

axs[1].set_title('% of days of using e-cigars and alike last month'),
axs[1].set_ylabel('percentage(%)')
axs[1].set_yticks(np.linspace(0,1,21),np.linspace(0,100,21))

axs[0].legend(days_smoke[1:],bbox_to_anchor=(2,-0.1),ncol=6)
plt.savefig('../fig/part_1_15_other.svg', bbox_inches='tight')


#### - Fig16 correlation between products

In [ ]:
fig, axs = plt.subplots(1,3,figsize = (16,4))
plt.suptitle('Correlation of smoking diffrent tobacco products')
plt.subplots_adjust(top=0.8)
sns.heatmap(df_boroughs5_from_2015[['q32','q35','q38']].corr(), annot=True,ax=axs[0])
axs[0].set_title('all answers')
axs[0].xaxis.set_ticklabels(['cigarettes', 'e-vapor','cigars'])
axs[0].yaxis.set_ticklabels(['cigarettes', 'e-vapor','cigars'])

sns.heatmap(df_boroughs5_from_2015[['q32_bin', 'q35_bin', 'q38_bin']].corr(), annot=True,ax=axs[1])
axs[1].set_title('binarized answers')
axs[1].xaxis.set_ticklabels(['cigarettes', 'e-vapor','cigars'])
axs[1].yaxis.set_ticklabels(['cigarettes', 'e-vapor','cigars'])

sns.heatmap(df_boroughs5_from_2015[(df_boroughs5_from_2015['q32']!=1) 
                       & (df_boroughs5_from_2015['q35']!=1) 
                       & (df_boroughs5_from_2015['q38']!=1)][['q32', 'q35', 'q38']].corr(), annot=True,ax=axs[2])
axs[2].set_title('answers among smokers')
axs[2].xaxis.set_ticklabels(['cigarettes', 'e-vapor','cigars'])
axs[2].yaxis.set_ticklabels(['cigarettes', 'e-vapor','cigars'])
plt.savefig('../fig/part_1_16_correlation.svg', bbox_inches='tight')

#### Comment
- The patterns of days of smoking e-vapor and cigars per month don't differ much between boroughs.
- Smoking regular cigarettes, e-vapor products and cigars and alike are not strongly correlated, which means that smoking one type of product doesn't imply smoking other two products, and smoking one product very often in the last month doesn't necessarily imply smoking other two also that often.
- The frequency of smoking different products among smokers are  higher correlated than that of among the whole population, indicating the possible predictability whithin smokers.

# School analysis
## 1. Check consistency of survey questions 2016-2019

#### 2016

In [ ]:
q2016 = "DBN;;School Name;;Total Student Response Rate;;1a. This school offers a wide enough variety of programs, classes and activities to keep me interested in school.;;1b. The programs, classes, and activities at this school encourage students to develop talent outside academics.;;1c. This school is kept clean.;;1d. Most students at this school treat each other with respect.;;1e. At this school students with disabilities are included in all school activities (lunch, class trips, etc.).;;1f. My teachers use examples of students’ different cultures/backgrounds/families in their lessons to make learning more meaningful for me.;;1g. I see people of many races, ethnicities, cultures, and backgrounds represented in the curriculum.;;1h. My teachers call on students of different races, ethnicities, cultures, and backgrounds.;;1i. I feel that my teachers respect my culture/background.;;2a. I learn a lot from feedback on my work.;;2b. It's clear what I need to do to get a good grade.;;2c. The work we do in class is good preparation for our class tests.;;2d. The homework assignments help me learn the course material.;;2e. I know what my teacher wants me to learn in class.;;3a. My teachers help me catch up if I am behind.;;3b. My teachers notice if I have trouble learning something.;;3c. My teachers give me specific suggestions about how I can improve my work in class.;;3d. My teachers explain things a different way if I don't understand something in class.;;3e. My teachers notice when I am upset.;;4a. In how many of your classes are you challenged?;;4b. In how many of your classes do your teachers ask difficult questions on tests?;;4c. In how many of your classes do your teachers ask difficult questions in class?;;4d. In how many of your classes do you work in small groups?;;4e. In how many of your classes do your teachers want students to become better thinkers, not just memorize things?;;5a. I’m learning a lot in my classes at this school to prepare me for the next level or grade.;;5b. There is at least one adult in the school that I can confide in.;;5c. My teachers will always listen to students' ideas.;;5d. My teachers always keep their promises.;;5e. My teachers treat me with respect.;;5f. When my teachers tell me not to do something, I know they have a good reason.;;5g. My classes at this school really make me think.;;5h. Discipline is applied fairly in my school.;;5i. School Safety Agents promote a safe and respectful environment at this school.;;6a. In how many of your classes at this school do students listen carefully when the teacher gives directions?;;6b. In how many of your classes at this school do students follow the rules in class?;;6c. In how many of your classes at this school do students pay attention when they are supposed to?;;6d. In how many of your classes at this school do students work when they are supposed to?;;6e. In how many of your classes at this school do students behave well even when the teacher isn't watching?;;6f. In how many of your classes at this school do students feel it is important to come to school every day?;;6g. In how many of your classes at this school do students feel it is important to pay attention in class?;;6h. In how many of your classes at this school do students think doing homework is important?;;6i. In how many of your classes at this school do students try hard to get good grades?;;7a. I feel safe outside around this school.;;7b. I feel safe traveling between home and this school.;;7c. I feel safe in the hallways, bathrooms, locker rooms, and cafeteria of this school.;;7d. I feel safe in my classes at this school.;;8a. At this school students harass, bully or intimidate other students.;;8b. At this school students harass, bully, or intimidate each other because of their race or ethnicity. ;;8c. At this school students harass, bully, or intimidate each other because of their gender, gender identity, gender expression, or sexual orientation.;;8d. At this school students harass, bully, or intimidate each other because of other differences, like national origin, citizenship/immigration status, religion, disability, or weight.;;8e. At this school students get into physical fights.;;8f. At this school students drink alcohol, use illegal drugs, or abuse prescription drugs while at school.;;8g. At this school there is gang activity.;;9a. If you are a student in grades 6-8, ANSWER this question. This school provides useful information to students about the application/enrollment process to high school.;;9b. If you are a student in grades 6-8, ANSWER this question. This school provides guidance for the application process for high school.;;9c. If you are a student in grades 6-8, ANSWER this question. This school educates families about the application/enrollment process for high school.;;10a. If you are a student in grades 9-12, ANSWER this question. Adults at this school (including teachers, administrators, counselors, and the principal) help keep me on track for college or career. ;;10b. If you are a student in grades 9-12, ANSWER this question. Adults at this school (including teachers, administrators, counselors, and the principal) provide me with information about the college enrollment process.;;10c. If you are a student in grades 9-12, ANSWER this question. Adults at this school (including teachers, administrators, counselors, and the principal) help me plan for my next steps after graduation (career planning, college selection and application process, financial aid process, etc.).;;10d. If you are a student in grades 9-12, ANSWER this question. Adults at this school (including teachers, administrators, counselors, and the principal) help me choose which colleges to apply to.;;10e. If you are a student in grades 9-12, ANSWER this question. Adults at this school (including teachers, administrators, counselors, and the principal) talk with me about how to pay for college. ;;10f. If you are a student in grades 9-12, ANSWER this question. Adults at this school (including teachers, administrators, counselors, and the principal) help me plan for how to meet my future career goals"
q2016 = q2016.split(';;')
# len(q2016)

In [ ]:
q2016

#### 2017

In [ ]:
q2017 = "DBN;;School Name;;Total Student Response Rate;;1a. This school offers a wide enough variety of programs, classes and activities to keep me interested in school.;;1b. The programs, classes, and activities at this school encourage students to develop talent outside academics.;;1c. This school is kept clean.;;1d. Most students at this school treat each other with respect.;;1e. At this school students with disabilities are included in all school activities (lunch, class trips, etc.).;;1f. My teachers use examples of students’ different cultures/backgrounds/families in their lessons to make learning more meaningful for me.;;1g. I see people of many races, ethnicities, cultures, and backgrounds represented in the curriculum.;;1h. In general, my teachers treat students from different cultures or backgrounds equally.;;1i. I feel that my teachers respect my culture/background.;;1j. In general, my teachers make their lessons relevant to my everday life experiences.;;1k. In general, my teachers present positive images of people from a variety of races, ethnicities, cultures, and backgrounds.;;2a. I learn a lot from feedback on my work.;;2b. It's clear what I need to do to get a good grade.;;2c. The work we do in class is good preparation for our class tests.;;2d. The homework assignments help me learn the course material.;;2e. I know what my teacher wants me to learn in class.;;3a. My teachers help me catch up if I am behind.;;3b. My teachers notice if I have trouble learning something.;;3c. My teachers give me specific suggestions about how I can improve my work in class.;;3d. My teachers explain things a different way if I don't understand something in class.;;3e. My teachers support me when I am upset.;;4a. I’m learning a lot in my classes at this school to prepare me for the next level or grade.;;4b. There is at least one adult in the school that I can confide in.;;4c. My teachers will always listen to students' ideas.;;4d. My teachers always do what they say they will do.;;4e. My teachers treat me with respect.;;4f. When my teachers tell me not to do something, I know they have a good reason.;;4g. My classes at this school really make me think critically.;;4h. Discipline is applied fairly in my school.;;4i. School Safety Agents promote a safe and respectful environment at this school.;;5a. In how many of your classes are you challenged?;;5b. In how many of your classes do your teachers ask difficult questions on tests?;;5c. In how many of your classes do your teachers ask difficult questions in class?;;5d. In how many of your classes do you work in small groups?;;5e. In how many of your classes do your teachers want students to become better thinkers, not just memorize things?;;6a. In how many of your classes at this school do YOU feel most students listen carefully when the teacher gives directions?;;6b. In how many of your classes at this school do YOU feel most students follow the rules in class?;;6c. In how many of your classes at this school do YOU feel most students pay attention when they are supposed to?;;6d. In how many of your classes at this school do YOU feel most students work when they are supposed to?;;6e. In how many of your classes at this school do YOU feel most students behave well even when the teacher isn't watching?;;6f. In how many of your classes at this school do YOU feel most students feel it is important to come to school every day?;;6g. In how many of your classes at this school do YOU feel most students feel it is important to pay attention in class?;;6h. In how many of your classes at this school do YOU feel most students think doing homework is important?;;6i. In how many of your classes at this school do YOU feel most students try hard to get good grades?;;7a. I feel safe outside around this school.;;7b. I feel safe traveling between home and this school.;;7c. I feel safe in the hallways, bathrooms, locker rooms, and cafeteria of this school.;;7d. I feel safe in my classes at this school.;;8a. At this school students harass, bully or intimidate other students.;;8b. At this school students harass, bully, or intimidate each other because of their race, religion, ethnicity, national origin, or citizenship/immigration status. ;;8c. At this school students harass, bully, or intimidate each other because of their gender, gender identity, gender expression, or sexual orientation.;;8d. At this school students harass, bully, or intimidate each other because of other differences, like disability or weight.;;8e. At this school students get into physical fights.;;8f. At this school students drink alcohol, use illegal drugs, or abuse prescription drugs while at school.;;8g. At this school there is gang activity.;;9a. If you are a student in grades 6-8, ANSWER this question. If you are a student in grades 9-12, SKIP this question. How much do YOU agree with the following statements? This school provides useful information to students about the application/enrollment process to high school.;;9b. If you are a student in grades 6-8, ANSWER this question. If you are a student in grades 9-12, SKIP this question. How much do YOU agree with the following statements? This school provides guidance for the application process for high school.;;9c. If you are a student in grades 6-8, ANSWER this question. If you are a student in grades 9-12, SKIP this question. How much do YOU agree with the following statements? This school educates families about the application/enrollment process for high school.;;10a.  If you are a student in grades 9-12, ANSWER this question. How much do YOU agree with the following statements? Adults at this school (including teachers, administrators, counselors, and the principal) encourage me to continue my education after high school. ;;10b.  If you are a student in grades 9-12, ANSWER this question. How much do YOU agree with the following statements? Adults at this school (including teachers, administrators, counselors, and the principal) provide me with information about the college enrollment process (college selection and application process, financial aid process, course registration, etc.).;;10c.  If you are a student in grades 9-12, ANSWER this question. How much do YOU agree with the following statements? Adults at this school (including teachers, administrators, counselors, and the principal) help me plan for how to meet my future career goals. ;;11a.  If you are a student in grades 9-12, ANSWER this question. How much do YOU agree with the following statements? Adults at this school (including teachers, administrators, counselors, and the principal) help me consider which colleges to apply to.;;11b.  If you are a student in grades 9-12, ANSWER this question. How much do YOU agree with the following statements? Adults at this school (including teachers, administrators, counselors, and the principal) show me options for how to pay for college (scholarship, grants, loans, work study programs, etc.)."
q2017 = q2017.split(';;')

a = list(set(q2016).intersection(q2017))
len(a)
a.sort()
a

In [ ]:
for i in q2016:
    if i not in a:
        print(i)

#### 2018

In [ ]:
q2018 = "DBN;;School Name;;Total Student Response Rate;;1a. This school offers a wide enough variety of programs, classes and activities to keep me interested in school.;;1b. The programs, classes, and activities at this school encourage students to develop talent outside academics.;;1c. This school is kept clean.;;1d. Most students at this school treat each other with respect.;;1e. At this school students with disabilities are included in all school activities (lunch, class trips, etc.).;;1f. My teachers use examples of students’ different cultures/backgrounds/families in their lessons to make learning more meaningful for me.;;1g. I see people of many races, ethnicities, cultures, and backgrounds represented in the curriculum.;;1h. In general, my teachers treat students from different cultures or backgrounds equally.;;1i. I feel that my teachers respect my culture/background.;;1j. In general, my teachers make their lessons relevant to my everday life experiences.;;1k. In general, my teachers present positive images of people from a variety of races, ethnicities, cultures, and backgrounds.;;2a. I learn a lot from feedback on my work.;;2b. It's clear what I need to do to get a good grade.;;2c. The work we do in class is good preparation for our class tests.;;2d. The homework assignments help me learn the course material.;;2e. I know what my teacher wants me to learn in class.;;3a. My teachers help me catch up if I am behind.;;3b. My teachers notice if I have trouble learning something.;;3c. My teachers give me specific suggestions about how I can improve my work in class.;;3d. My teachers explain things a different way if I don't understand something in class.;;3e. My teachers support me when I am upset.;;4a. I’m learning a lot in my classes at this school to prepare me for the next level or grade.;;4b. There is at least one adult in the school that I can confide in.;;4c. My teachers will always listen to students' ideas.;;4d. My teachers always do what they say they will do.;;4e. My teachers treat me with respect.;;4f. When my teachers tell me not to do something, I know they have a good reason.;;4g. My classes at this school really make me think critically.;;4h. Discipline is applied fairly in my school.;;4i. School Safety Agents promote a safe and respectful environment at this school.;;5a. In how many of your classes are you challenged?;;5b. In how many of your classes do your teachers ask difficult questions on tests?;;5c. In how many of your classes do your teachers ask difficult questions in class?;;5d. In how many of your classes do you work in small groups?;;5e. In how many of your classes do your teachers want students to become better thinkers, not just memorize things?;;6a. In how many of your classes at this school do YOU feel most students listen carefully when the teacher gives directions?;;6b. In how many of your classes at this school do YOU feel most students follow the rules in class?;;6c. In how many of your classes at this school do YOU feel most students pay attention when they are supposed to?;;6d. In how many of your classes at this school do YOU feel most students work when they are supposed to?;;6e. In how many of your classes at this school do YOU feel most students behave well even when the teacher isn't watching?;;6f. In how many of your classes at this school do YOU feel most students feel it is important to come to school every day?;;6g. In how many of your classes at this school do YOU feel most students feel it is important to pay attention in class?;;6h. In how many of your classes at this school do YOU feel most students think doing homework is important?;;6i. In how many of your classes at this school do YOU feel most students try hard to get good grades?;;7a. I feel safe outside around this school.;;7b. I feel safe traveling between home and this school.;;7c. I feel safe in the hallways, bathrooms, locker rooms, and cafeteria of this school.;;7d. I feel safe in my classes at this school.;;8a. At this school students harass, bully or intimidate other students.;;8b. At this school students harass, bully, or intimidate each other because of their race, religion, ethnicity, national origin, or citizenship/immigration status. ;;8c. At this school students harass, bully, or intimidate each other because of their gender, gender identity, gender expression, or sexual orientation.;;8d. At this school students harass, bully, or intimidate each other because of other differences, like disability or weight.;;8e. At this school students get into physical fights.;;8f. At this school students drink alcohol, use illegal drugs, or abuse prescription drugs while at school.;;8g. At this school there is gang activity.;;9a. For Middle School: This school provides guidance for the application process for high school.;;9b. For Middle School: This school educates families about the application/enrollment process for high school.;;10a. For High School: Adults at this school (including teachers, administrators, counselors, and the principal) talk to me about what I plan to do after high school.;;10b. For High School: Adults at this school (including teachers, administrators, counselors, and the principal) encourage me to continue my education after high school.;;10c. For High School: Adults at this school (including teachers, administrators, counselors, and the principal) provide me with information about the college application process.;;10d. For High School: Adults at this school (including teachers, administrators, counselors, and the principal) help me plan for how to meet my future career goals. ;;11a. For High School: Adults at this school (including teachers, administrators, counselors, and the principal) help me consider which colleges to apply to.;;11b. For High School: Adults at this school (including teachers, administrators, counselors, and the principal) show me options for how to pay for college (scholarship, grants, loans, work study programs, etc.)."
q2018 = q2018.split(";;")

In [ ]:
q2018

In [ ]:
b = list(set(q2017).intersection(q2018))
b.sort()
b

In [ ]:
for i in q2017:
    if i not in b:
        print(i)

In [ ]:
for i in q2018:
    if i not in b:
        print(i)

#### 2019

In [ ]:
q2019 = "DBN;;School Name;;Total Student Response Rate;;1a. This school offers a wide enough variety of programs, classes and activities to keep me interested in school.;;1b. The programs, classes, and activities at this school encourage students to develop talent outside academics.;;1c. This school is kept clean.;;1d. Most students at this school treat each other with respect.;;1e. At this school, students with disabilities are included in all school activities (lunch, class trips, etc.).;;1f. My teachers use examples of students’ different cultures/backgrounds/families in their lessons to make learning more meaningful for me.;;1g. I see people of many races, ethnicities, cultures, and backgrounds represented in the curriculum.;;1h. In general, my teachers treat students from different cultures or backgrounds equally.;;1i. I feel that my teachers respect my culture/background.;;1j. In general, my teachers make their lessons relevant to my everyday life experiences.;;1k. In general, my teachers present positive images of people from a variety of races, ethnicities, cultures, and backgrounds.;;2a. I learn a lot from feedback on my work.;;2b. It's clear what I need to do to get a good grade.;;2c. The work we do in class is good preparation for our class tests.;;2d. The homework assignments help me learn the course material.;;2e. I know what my teacher wants me to learn in class.;;3a. My teachers help me catch up if I am behind.;;3b. My teachers notice if I have trouble learning something.;;3c. My teachers give me specific suggestions about how I can improve my work in class.;;3d. My teachers explain things a different way if I don't understand something in class.;;3e. My teachers support me when I am upset.;;4a. I’m learning a lot in my classes at this school to prepare me for the next level or grade.;;4b. There is at least one adult in the school that I can confide in.;;4c. My teachers will always listen to students' ideas.;;4d. My teachers always do what they say they will do.;;4e. My teachers treat me with respect.;;4f. When my teachers tell me not to do something, I know they have a good reason.;;4g. My classes at this school really make me think critically.;;4h. Discipline is applied fairly in my school.;;4i. School Safety Agents promote a safe and respectful environment at this school.;;5a. In how many of your classes are you challenged?;;5b. In how many of your classes do your teachers ask difficult questions on tests?;;5c. In how many of your classes do your teachers ask difficult questions in class?;;5d. In how many of your classes do you work in small groups?;;5e. In how many of your classes do your teachers want students to become better thinkers, not just memorize things?;;6a. In how many of your classes at this school do YOU feel most students listen carefully when the teacher gives directions?;;6b. In how many of your classes at this school do YOU feel most students follow the rules in class?;;6c. In how many of your classes at this school do YOU feel most students pay attention when they are supposed to?;;6d. In how many of your classes at this school do YOU feel most students work when they are supposed to?;;6e. In how many of your classes at this school do YOU feel most students behave well even when the teacher isn't watching?;;6f. In how many of your classes at this school do YOU feel most students feel it is important to come to school every day?;;6g. In how many of your classes at this school do YOU feel most students feel it is important to pay attention in class?;;6h. In how many of your classes at this school do YOU feel most students think doing homework is important?;;6i. In how many of your classes at this school do YOU feel most students try hard to get good grades?;;7a. I feel safe outside around this school.;;7b. I feel safe traveling between home and this school.;;7c. I feel safe in the hallways, bathrooms, locker rooms, and cafeteria of this school.;;7d. I feel safe in my classes at this school.;;8a. At this school students harass, bully or intimidate other students.;;8b. At this school students harass, bully, or intimidate each other because of their race, religion, ethnicity, national origin, or citizenship/immigration status. ;;8c. At this school students harass, bully, or intimidate each other because of their gender, gender identity, gender expression, or sexual orientation.;;8d. At this school students harass, bully, or intimidate each other because of other differences, like disability or weight.;;8e. At this school students harass, bully, or intimidate each other online (through mobile phones, social media, email, or other forms of electronic communication). ;;8f. At this school students get into physical fights.;;8g. At this school students drink alcohol, use illegal drugs, or abuse prescription drugs while at school.;;8h. At this school there is gang activity.;;9a. For Middle School: This school provides guidance for the application process for high school.;;9b. For Middle School: This school educates families about the application/enrollment process for high school.;;10a. For High School: Adults at this school (including teachers, administrators, counselors, and the principal) talk to me about what I plan to do after high school.;;10b. For High School: Adults at this school (including teachers, administrators, counselors, and the principal) encourage me to continue my education after high school.;;10c. For High School: Adults at this school (including teachers, administrators, counselors, and the principal) provide me with information about the college application process.;;10d.   For High School: Adults at this school (including teachers, administrators, counselors, and the principal) help me plan for how to meet my future career goals. ;;11a. For High School: Adults at this school (including teachers, administrators, counselors, and the principal) help me consider which colleges to apply to.;;11b. For High School: Adults at this school (including teachers, administrators, counselors, and the principal) show me options for how to pay for college (scholarship, grants, loans, work study programs, etc.)."
q2019 = q2019.split(';;')
c = list(set(b).intersection(q2019))
c.sort()

In [ ]:
q2019

In [ ]:
c

In [ ]:
for i in b:
    if i not in c:
        print(i)

## 2. Read the dataframes and concatenate into one big dataframe

#### 2017&2018

In [ ]:
df16 = pd.DataFrame()
df17 = pd.DataFrame()
df18 = pd.DataFrame()
df19 = pd.DataFrame()

In [ ]:
df17 = pd.read_table("./data/2017student.csv", sep=',',header=[0,1]).iloc[:,:234]
df18 = pd.read_table("./data/2018student.csv", sep=',',header=[0,1]).iloc[:,:234]

In [ ]:
questions = [None] * len(df17.columns)

for i in range(len(questions)):
    if 'Unnamed' not in list(df17.columns)[i][0]:
        questions[i] = list(df17.columns)[i][0]
        
mask = np.array([elem is not None for elem in questions])

temp = None
for i,flag in enumerate(mask):
    if flag:
        temp = questions[i]
    else:
        questions[i] = temp

for i,q in enumerate(questions):
    if '8e' in q:
        questions[i] = questions[i].replace('8e','8f')
    if '8f' in q:
        questions[i] = questions[i].replace('8f','8g')
    if '8g' in q:
        questions[i] = questions[i].replace('8g','8h')
        
index_q_a = questions[:3]+  [questions[i][:2] + ': ' + list(df17.columns)[i][1] for i in range(3,len(questions))]

In [ ]:
df17.columns = index_q_a
df18.columns = index_q_a

#### 2016

In [ ]:
df16 = pd.read_table("./data/2016student.csv", sep=',',header=[0,1]).iloc[:,:224]
q16 = [i[0] for i in list(df16.columns)]
mask = np.array(['Unnamed' not in elem for elem in q16])

In [ ]:
temp = None
for i,q in enumerate(q16):
    if mask[i]:
        temp = q
    else:
        q16[i] = temp
        
for i,q in enumerate(q16):
    if '8e' in q:
        q16[i]='8f'
    if '8f' in q:
        q16[i]='8g'
    if '8g' in q:
        q16[i]='8h' 
        
    if '4' in q:
        q16[i] = q.replace('4','5')
    if '5' in q:
        q16[i] = q.replace('5','4')
        
index_q_a_16 = q16[:3]+  [q16[i][:2] + ': ' + list(df16.columns)[i][1] for i in range(3,len(q16))]
df16.columns = index_q_a_16
df16 = df16[~df16['DBN'].isnull()]

#### 2019

In [ ]:
df19 = pd.read_table("./data/2019student.csv", sep=',',header=[0,1]).iloc[:,:238]
q19 = [i[0] for i in list(df19.columns)]
mask = np.array(['Unnamed' not in ele for ele in q19])

In [ ]:
question_dict = dict()

temp = None
for i,q in enumerate(q19):
    if mask[i]:
        if i>=3:
            question_dict[q[:2]] = q[4:]
        temp = q
    else:
        q19[i] = temp 

index_q_a_19 = q19[:3]+  [q19[i][:2] + ': ' + list(df19.columns)[i][1] for i in range(3,len(q19))]
df19.columns = index_q_a_19


In [ ]:
question_dict

#### concatenate with year

In [ ]:
df17['year'] = 2017
df18['year'] = 2018
df16['year'] = 2016
df19['year'] = 2019

In [ ]:
#df_schools = pd.concat((df16,df17))
df_schools = pd.concat((df17,df18))
df_schools = pd.concat((df_schools,df19))

In [ ]:
df_schools

## 3. Read the school locations and filter only high schools

In [ ]:
df_school_directory = pd.read_table("./data/2019_DOE_High_School_Directory.csv", sep=',')[['dbn','school_name','boro','Borough','Latitude','Longitude']]

df_school_directory = df_school_directory.rename(columns={'dbn':'DBN'})
df_school_directory.replace(df_school_directory.Borough.unique(), ['Brooklyn', 'Manhattan', 'Queens', 'Bronx', 'Staten Island'], inplace=True)#.groupby('Borough').count()
df_high_schools = pd.merge(df_schools,df_school_directory,'right')

df_school_directory

In [ ]:
geodata_path = "NYC.geojson";
with open(geodata_path) as f:
    geojson = json.load(f)

geojson_Queens = geojson['features'][0]['geometry'];
geojson_Brooklyn = geojson['features'][1]['geometry'];
geojson_Bronx = geojson['features'][4]['geometry'];
geojson_Manhattan = geojson['features'][3]['geometry'];
geojson_StatenIsland = geojson['features'][2]['geometry'];

geo_dict = {'Borough': ['Queens', 'Brooklyn', 'Bronx', 'Manhattan', 'StatenIsland'],
     'geometry': [shape(geojson_Queens), shape(geojson_Brooklyn), shape(geojson_Bronx), shape(geojson_Manhattan),
                  shape(geojson_StatenIsland)]};
gdf_borough = gpd.GeoDataFrame(geo_dict).set_index("Borough");

gdf_borough

In [ ]:
nyc_population_borough = [{ 'Borough':'Queens', 'Population':2405464 }, { 'Borough':'Brooklyn', 'Population':2736074 }, { 'Borough':'Staten Island', 'Population':495747 }, { 'Borough':'Manhattan', 'Population':1694263 }, { 'Borough':'Bronx', 'Population':1472654 }];

df_nyc_borough_population = pd.DataFrame(data=nyc_population_borough, columns={'Borough', 'Population'})\
    .set_index("Borough");

df_nyc_borough_population = df_nyc_borough_population.squeeze()
df_nyc_borough_population

In [ ]:
df_school_per_100k_borough = df_school_directory.value_counts("Borough").div(df_nyc_borough_population).mul(100000)
df_school_per_100k_borough
# ["data"].div(df_nyc_borough_population["population"])

In [ ]:
cityhall_lat_lon = [40.7128, -74.0060];
map_nyc_school = folium.Map(location=cityhall_lat_lon,
                    tiles = "OpenStreetMap",
                    zoom_start = 11);

folium.TileLayer('Stamen Toner').add_to(map_nyc_school)
folium.TileLayer('cartodbpositron').add_to(map_nyc_school)

### create markers
marker_layer = FeatureGroup(name='Markers for high schools')
for (index, row) in df_school_directory.iterrows():
    marker_layer.add_child(folium.Marker([row["Latitude"], row["Longitude"]], popup=row["school_name"],
                  icon=folium.Icon(color='green', prefix='fa', icon='graduation-cap')))

map_nyc_school.add_child(marker_layer)

### Distributions of high schools in each borough
folium.Choropleth(
    geo_data=geojson,
    name="Distributions of high schools in each borough",
    data=df_school_directory.value_counts("Borough"),
    columns=["Borough"],
    key_on="properties.boro_name",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Number of Schools",
).add_to(map_nyc_school)

### Distributions of population in each borough
folium.Choropleth(
    geo_data=geojson,
    name="Distributions of population in each borough",
    data=df_nyc_borough_population,
    columns=["Borough"],
    key_on="properties.boro_name",
    fill_color="Blues",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Population",
    show=False
).add_to(map_nyc_school)

### Distributions of school in 100k population in each borough
folium.Choropleth(
    geo_data=geojson,
    name="Distributions of school in 100k population in each borough",
    data=df_school_per_100k_borough,
    columns=["Borough"],
    key_on="properties.boro_name",
    fill_color="Purples",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="School per 100k people in each borough",
    show=False
).add_to(map_nyc_school)

### Borough borders Plot
folium.features.GeoJson(name="Borough borders", data=geojson,
                        style_function=lambda x :{'color': "white", "weight": 2},
                        show=True).add_to(map_nyc_school)

folium.LayerControl().add_to(map_nyc_school)

map_nyc_school

In [ ]:
map_nyc_school.save("../fig/part_2_high_school_overview.html")

### COMMENTS
An overview for distributions of high schools and population in NYC with respect to five boroughs. You can play with
the layers using the button on the right top corner. With outstanding amount of schools as well as decent population,
 Bronx and Manhattan make themselves the best boroughs when comes to the number of school per capita in the borough.
 Things become a little messy in Queens and Brooklyn. Given top population among boroughs, citizens may have to live
 with the fact that getting education nearby would be challenging due to fair expectations on competition.

In [ ]:
def geoplot_answer(map, df) :
    folium.TileLayer('Stamen Toner').add_to(map)
    is_first = True;

    for (col_name, col_data) in df.iteritems():
        if (col_name == "Borough") :
            continue;

        folium.Choropleth(
            geo_data=geojson,
            name="Answer distribution for option " + str(col_name),
            data=df,
            columns=["Borough", col_name],
            key_on="properties.boro_name",
            fill_color="OrRd",
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name="Answer rate for option " + str(col_name),
            show=is_first
        ).add_to(map)

        is_first = False;

    folium.features.GeoJson(name="Borough borders", data=geojson,
                        style_function=lambda x :{'color': "white", "weight": 2},
                        show=True).add_to(map)

    folium.LayerControl().add_to(map)

### visualize the schools amount

In [ ]:
school_borough = df_school_directory.groupby('Borough').count()['DBN'].to_frame()
school_borough

In [ ]:
plt.figure(figsize=(10, 6), dpi=300)
plt.title("High School Distribution in Each Borough", fontsize=16)
plt.ylabel("Count")
school_borough.plot(kind='bar', ax=plt.gca())

plt.savefig('../fig/part_2_high_school_distribution_borough.svg')

## 4. School scoring

### 4.1 framework:  six element
Rigorous Instruction, 
Collaborative Teachers, Supportive Environment, Effective School Leadership, 
Strong Family-Community Ties, and Trust

### 4.2 methodology:
https://infohub.nyced.org/docs/default-source/default-document-library/framework-school-survey-scoring-technical-guide.pdf

The following process is used to generate a survey element score:  
(1)  Question-level percent positive (percentage of positive responses to a 
question)  
(2)  Measure-level percent positive (average of the question-level percent 
positive values for all questions within the measure)   
(3)  Measure score (score based on the measure-level percent positive)
> scaling method:  
    **Rating Category | Percent  Positive (PP) Cut Level**  
    4.99 Top of Scoring Range |
citywide mean + 2 SD, not to exceed 100  
    4.00-4.99 Exceeding Target (4 bars) | 
citywide mean PP + 0.75 SD, not to exceed 95   
    3.00-3.99 Meeting Target (3 bars) |
citywide mean PP – 0.5 SD, not to exceed 90  
    2.00-2.99 Approaching Target (2 bars) | citywide mean PP – 1 SD, not to exceed 85  
    1.00-1.99 Bottom of Scoring Range | citywide mean + 2 SD, not to fall below 0  
    
(4)  Survey element score (average of measure scores for all measures within 
the element)



### 4.3 choose element:  Supportive Environment

- M1 Classroom behavior  
q6a q6b q6c q6d q6e
- M2 Guidance  
q9a q9b q10a q10b q10c q10d q11a q11b  
- M3 Peer support for academic work  
q6f q6g q6h q6i 
- M4 Personal attention and support  
q3a q3b q3c q3d q3e 
- M5 Preventing bullying  
q8a q8b q8c q8d q8e 
- M6 Safety  
q4h q7a q7b q7c q7d 

### 4.4 Average scores through 2017-2019 of each school

In [ ]:
scoring_set = {'M1':['6a', '6b', '6c', '6d', '6e'],
               'M3':['6f', '6g', '6h', '6i'],
               'M4':['3a', '3b', '3c', '3d', '3e'],
               'M5':['8a', '8b', '8c', '8d', '8e'],
               'M6':['4h', '7a', '7b', '7c', '7d']
              }

# check the answers
for group,qs in scoring_set.items():
    print('____________ measure group '+group+' _____________')
    for idx in qs:
        print(idx+':', question_dict[idx])
        print([s for s in np.array(df_high_schools.columns, dtype=str) if idx in s],'\n')

In [ ]:
# calculate each schools positive rate of each question and the avg of each measure
for group,measure in scoring_set.items():
    for idx in measure:
        if ('3' in idx) or ('6' in idx) or ('4' in idx) or ('7' in idx): 
            df_answers = df_high_schools[[s for s in np.array(df_high_schools.columns, dtype=str) if idx in s]]
            df_high_schools[idx+'_pos'] = df_answers.iloc[:,-2:].sum(axis = 1)/df_answers.sum(axis = 1) 
        if '8' in idx:
            df_answers = df_high_schools[[s for s in np.array(df_high_schools.columns, dtype=str) if idx in s]]
            df_high_schools[idx+'_pos'] = df_answers.iloc[:,:2].sum(axis = 1)/df_answers.sum(axis = 1) 
        
    df_high_schools[group+'_avg'] = df_high_schools[[idx + '_pos'for idx in measure]].mean(axis = 1)
            

In [ ]:
# draw the distribution of average positive rate of each measure
plt.figure(figsize=(10, 6), dpi=300)
plt.boxplot([df_high_schools[i + '_avg'].dropna().values for i in list(scoring_set.keys())])

In [ ]:
# scale to scores
for group, measure in scoring_set.items():
    
    col = group + '_score'
    # print(col)
    df_high_schools[col] = np.nan
    
    mean = df_high_schools[group + '_avg'].dropna().mean()
    std =  df_high_schools[group + '_avg'].dropna().std()
    
    bar5 = min(mean + 2*std, 1)
    bar4 = min(mean + 0.75*std, 0.95)
    bar3 = min(mean - 0.5*std, 0.90)
    bar2 = min(mean - std, 0.85)
    bar1 = min(mean - 2*std, 0)
    
    df_high_schools.loc[df_high_schools[group + '_avg'] > bar5, col] = 4.99
    df_high_schools.loc[df_high_schools[group + '_avg'] < bar5, col] = 1.00
    
    mask = (df_high_schools[group + '_avg'] < bar5) & (df_high_schools[group + '_avg'] > bar4)
    sub_4 = df_high_schools[mask]
    df_high_schools.loc[mask,col] = 4 + (sub_4[group + '_avg']-bar4)/(bar5-bar4)
    
    mask = (df_high_schools[group + '_avg'] < bar4) & (df_high_schools[group + '_avg'] > bar3)
    sub_3 = df_high_schools[mask]
    df_high_schools.loc[mask,col] = 3 + (sub_3[group + '_avg']-bar3)/(bar4-bar3)

    
    mask = (df_high_schools[group + '_avg'] < bar3) & (df_high_schools[group + '_avg'] > bar2)
    sub_2 = df_high_schools[mask]
    df_high_schools.loc[mask,col] = 2 + (sub_2[group + '_avg']-bar2)/(bar3-bar2)

    
    mask = (df_high_schools[group + '_avg'] < bar2) & (df_high_schools[group + '_avg'] > bar1)
    sub_1 = df_high_schools[mask]
    df_high_schools.loc[mask,col] = 1 + (sub_1[group + '_avg']-bar1)/(bar2-bar1)


In [ ]:
# average score of "supportive environment"
df_high_schools['element_score'] = df_high_schools[['M1_score','M3_score','M4_score','M5_score','M6_score']].mean(axis=1)

In [ ]:
# visualize the element scores of each borough
score_boroughs = []
boroughs = boroughs_name
for b in boroughs_name:
    score_boroughs.append(df_high_schools[df_high_schools['Borough']==b]['element_score'].dropna())


plt.figure(figsize=(10, 6), dpi=300)
fig = plt.boxplot(score_boroughs,labels=boroughs_name)

In [ ]:
row_num = 3;
col_num = 2;

fig, axes = plt.subplots(row_num, col_num, figsize=(10, 6), sharey=True, constrained_layout=True, dpi=300)
fig.suptitle('Number of votes for each option in supportive environment', fontsize=16)

for (idx, b) in enumerate(boroughs_name):
    # plt.figure()
    # plt.title(b)
    ax = axes[idx//col_num, idx%col_num];
    sns.histplot(ax=ax, data=df_high_schools[df_high_schools['Borough']==b].element_score);
    ax.set_title(b, loc="left");

axes[row_num-1, col_num-1].remove();
fig.savefig('../fig/part_2_score_on_items.svg')

### 4.5 yearly changes

In [ ]:
year_score_boroughs = dict()
for y in [2017, 2018, 2019]:
    year_score_boroughs[y] = []
    df_year = df_high_schools[df_high_schools['year']==y]
    for b in boroughs:
        year_score_boroughs[y].append(df_year[df_year['Borough']==b]['element_score'].dropna())   

In [ ]:
year_score_boroughs_avg = pd.DataFrame(columns=boroughs,index=[2017,2018,2019])
for y in [2017, 2018, 2019]:
    df_year = df_high_schools[df_high_schools['year']==y]
    for b in boroughs:
        year_score_boroughs_avg.loc[y,b] = df_year[df_year['Borough']==b]['element_score'].dropna().mean()

In [ ]:
year_score_boroughs_avg

In [ ]:
def geoplot_time_series(map, df, colors, title) :
    # folium.TileLayer('openstreetmap').add_to(map);
    # folium.TileLayer('Stamen Toner').add_to(map);

    # colorscale = branca.colormap.linear.YlGnBu_09.scale(0, 30);
    df = df.set_index("Borough");

    max_color = df.to_numpy().max();
    min_color = df.to_numpy().min();
    cmap = colors.scale(min_color, max_color);

    df = df.reset_index();

    df_color = df.copy()
    for (col_name, col_data) in df.iteritems():
            if (col_name == "Borough") :
                continue;

            df_color[col_name] = df[col_name].map(cmap)

    style_dict = {};

    for (_, row) in df_color.iterrows():
        inner_dict = {};

        for (col_name, col_data) in row.iteritems():
            if (col_name == "Borough") :
                continue;

            inner_dict[
                (int)((datetime.datetime(year=int(col_name), month=1,day=1)).timestamp())] = \
                                { 'color': row[int(col_name)], 'opacity': 0.7 };

        style_dict[row["Borough"].replace(' ', '')] = inner_dict;

    print(gdf_borough)
    print(style_dict)
    map.add_child(
        TimeSliderChoropleth(
            data=gdf_borough.to_json(), styledict=style_dict, overlay=True
        )
    );

    _ = cmap.add_to(map);
    cmap.caption = title;


In [ ]:
cityhall_lat_lon = [40.7128, -74.0060];
map = folium.Map(location=cityhall_lat_lon,
                    tiles = "openstreetmap",
                    zoom_start = 11);

df = year_score_boroughs_avg.transpose().reset_index().rename(columns={"index": "Borough"});

colors = branca.colormap.linear.YlGnBu_09;
title = "Yearly change of school score"
geoplot_time_series(map, df, colors, title);
map

In [ ]:
map.save("../fig/part_2_school_score_yearly_change.html")

In [ ]:
row_num = 2;
col_num = 2;

fig, axes = plt.subplots(row_num, col_num, figsize=(10, 6), sharey=True, constrained_layout=True, dpi=300)
fig.suptitle('Initial Pokemon - 1st Generation')

for (idx, y) in enumerate([2017, 2018, 2019]):
    ax = axes[idx//col_num, idx%col_num];
    ax.boxplot(x=year_score_boroughs[y], labels=boroughs)

axes[row_num-1, col_num-1].remove();

## 5. Comprehensive analysis with tobacco retailers 
### 5.1 import the retailers

In [ ]:
tobacco_retailers = pd.read_table('./data/Active_Tobacco_Retail_Dealer_Licenses.csv',sep=',')
tobacco_retailers['License Creation Date'] = pd.to_datetime(tobacco_retailers['License Creation Date'])
tobacco_retailers = tobacco_retailers[tobacco_retailers['License Creation Date'].dt.year<2021]
tobacco_retailers = tobacco_retailers[['DCA License Number','Address Borough','Borough Code','Longitude','Latitude','Location']]

print(len(tobacco_retailers))
print(tobacco_retailers['Address Borough'].unique())

In [ ]:
tobacco_retailers.loc[tobacco_retailers['Borough Code']==1, 'Address Borough'] = 'Manhattan'
tobacco_retailers.loc[tobacco_retailers['Borough Code']==4, 'Address Borough'] = 'Queens'

tobacco_retailers = tobacco_retailers.reset_index().drop(columns=['index'])
tobacco_retailers

In [ ]:
tobacco_retailers_count = tobacco_retailers.groupby('Address Borough').count()
tobacco_retailers_count

In [ ]:
plt.figure(figsize=(10, 6), dpi=300)
plt.title("Registered Tobacco Retailer Distribution in Each Borough", fontsize=16)
plt.ylabel("Count")
tobacco_retailers_count.plot(kind='bar', ax=plt.gca(), y="DCA License Number")

plt.savefig('../fig/part_2_retailer_distribution_borough.svg')

In [ ]:
tobacco_retailers_count_per_capita_100k = tobacco_retailers_count['DCA License Number'].div\
    (df_nyc_borough_population).mul(100000);
tobacco_retailers_count_per_capita_100k

In [ ]:
cityhall_lat_lon = [40.7128, -74.0060];
map_nyc_retail = folium.Map(location=cityhall_lat_lon,
                    tiles = "openstreetmap",
                    zoom_start = 11);

folium.TileLayer('Stamen Toner').add_to(map_nyc_retail)
folium.TileLayer('cartodbpositron').add_to(map_nyc_retail)

### Distributions of retailers in each borough
folium.Choropleth(
    geo_data=geojson,
    name="Distributions of retailers in each borough",
    data=tobacco_retailers_count.reset_index(),
    columns=["Address Borough", "DCA License Number"],
    key_on="properties.boro_name",
    fill_color="Reds",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Number of Retailers",
    show=False
).add_to(map_nyc_retail)

### Distributions of retailers in 100k population in each borough
folium.Choropleth(
    geo_data=geojson,
    name="Distributions of retailers in 100k population in each borough",
    data=tobacco_retailers_count_per_capita_100k,
    columns=[0],
    key_on="properties.boro_name",
    fill_color="BuPu",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Retailer per 100k people in each borough",
    show=False
).add_to(map_nyc_retail)

### heatmap of retailers
HeatMap(([row['Latitude'], row['Longitude']] for (idx, row) in tobacco_retailers.iterrows() if
         (not pd.isnull(row['Latitude'])) & (not pd.isnull(row['Longitude']))),
        radius=10, blur=10, name="Heatmap of retailers").add_to(map_nyc_retail);

### Borough borders Plot
folium.features.GeoJson(name="Borough borders", data=geojson,
                        style_function=lambda x :{'color': "white", "weight": 2},
                        show=True).add_to(map_nyc_retail)

folium.LayerControl().add_to(map_nyc_retail)

map_nyc_retail

In [ ]:
map_nyc_retail.save("../fig/part_2_retailer_overview.html")

### COMMENTS
When comes to number of retailers, it is not surprised that NYC has a huge amount of tobacco retailers in total. You
can click the button on top-right of the map for different information related to retailers. By default, you will be
looking at the heatmap describing the retailer distribution.
Given the knowledge that Manhattan ranks as the most famous and recognizable borough in NYC with Wall Street lies on,
 this borough enjoys the most intense distribution of legal tobacco retailers, seen from both heatmap and per 100k
 capita plot. Bronx, with the highest crime rate in NYC, keeps its population mild, hence a rather lower density for
 retailers in this borough than you might expect. This phenomenon also shows less relation between the crime rate
 with legal tobacco retailer density.

### 5.2 retailer:school ratio

In [ ]:
retailer_density = pd.DataFrame(index=tobacco_retailers_count.index,columns=['retailer:school ratio'])
for i in range(5):
    retailer_density.iloc[i,0] = tobacco_retailers_count.iloc[i,0]/school_borough.iloc[i,0]
retailer_density

### 5.3 distance to retailers: 
- for each shool find 5 nearest retailers
- calculate the average distance?
- (optional:tag the retailers that are shared the most between schools)

In [ ]:
from ast import literal_eval
tobacco_retailers.loc[:,'Location'] = tobacco_retailers.Location.dropna().apply(lambda x: literal_eval(str(x)))
#df_school_directory

In [ ]:
#df_school_directory['Borough'].replace(df_school_directory.Borough.unique(),retailer_density.index, inplace=True)

df_school_directory['nearest retailers idx'] = -1
df_school_directory['nearest retailers idx']  = df_school_directory['nearest retailers idx'].astype(object)
df_school_directory['nearest retailers distance']= -1
df_school_directory['nearest retailers distance']  = df_school_directory['nearest retailers distance'].astype(object)

df_school_directory['mean distance'] = -1

In [ ]:
# find the nearest retailers of each school
boroughs = df_school_directory.Borough.unique()

school_borough_dict = dict()

distance = []
nearest_retailers = []

for b in boroughs:
    
    school_in_borough = df_school_directory[df_school_directory.Borough==b]
    school_in_borough_index = school_in_borough.index
    tobacco_retailers = tobacco_retailers.dropna(subset = ['Location'])#tobacco_retailers[tobacco_retailers['Address Borough']==b].dropna(subset = ['Location'])
    
    for i in range(len(school_in_borough)):
        school_idx = school_in_borough_index[i]
        school_coordinates = (school_in_borough.iloc[i].Latitude, school_in_borough.iloc[i].Longitude)
        # print(school_coordinates)
        tobacco_retailers['distance'] = tobacco_retailers.Location.apply(
             lambda x: great_circle(school_coordinates, x).km)
        
        nearest_retailers = tobacco_retailers.sort_values('distance').iloc[:5]
        df_school_directory['nearest retailers idx'].iloc[school_idx] = np.array(nearest_retailers.index)
        df_school_directory['nearest retailers distance'].iloc[school_idx] = nearest_retailers.distance.values
        df_school_directory['mean distance'].iloc[school_idx] = nearest_retailers.distance.values.mean()

In [ ]:
df_school_directory

In [ ]:
cityhall_lat_lon = [40.7128, -74.0060];
map_nyc_retail_school = folium.Map(location=cityhall_lat_lon,
                    zoom_start = 11);

folium.TileLayer('Stamen Toner').add_to(map_nyc_retail_school)
folium.TileLayer('cartodbpositron').add_to(map_nyc_retail_school)

### create markers for schools
marker_layer_school = FeatureGroup(name='Markers for high schools', show=False)
for (index, row) in df_school_directory.iterrows():
    marker_layer_school.add_child(folium.Marker([row["Latitude"], row["Longitude"]], popup=row["school_name"],
                  icon=folium.Icon(color='green', prefix='fa', icon='graduation-cap')))

map_nyc_retail_school.add_child(marker_layer_school)

### heatmap of retailers
HeatMap(([row['Latitude'], row['Longitude']] for (idx, row) in tobacco_retailers.iterrows() if
         (not pd.isnull(row['Latitude'])) & (not pd.isnull(row['Longitude']))),
        radius=10, blur=10, name="Heatmap of retailers", show=False).add_to(map_nyc_retail_school);

### Distributions of ratio between number of retailer and high school in each borough
folium.Choropleth(
    geo_data=geojson,
    name="Distributions of ratio between number of retailer and high school in each borough",
    data=retailer_density.squeeze(),
    columns=["Address Borough"],
    key_on="properties.boro_name",
    fill_color="YlOrBr",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Ratio between number of retailer and high school",
    show=True
).add_to(map_nyc_retail_school)

### Marker layer for high school-to-retailer mean distance
marker_layer_dist = FeatureGroup(name='Markers for high school-to-retailer mean distance')
for (index, row) in df_school_directory.iterrows():
    marker_layer_dist.add_child(folium.Circle(location=[row["Latitude"], row["Longitude"]],
                                                    radius=row['mean distance']*1000,
                                                    popup=str(row["school_name"])+", mean distance = "+
                                                          str(round(row['mean distance']*1000))+"m",
                                                    color='#636363',
                                                    fill=True,
                                                    fill_color='#636363',
                                                    weight='2'));

map_nyc_retail_school.add_child(marker_layer_dist)

### Borough borders Plot
folium.features.GeoJson(name="Borough borders", data=geojson,
                        style_function=lambda x :{'color': "white", "weight": 2},
                        show=True).add_to(map_nyc_retail_school)

folium.LayerControl().add_to(map_nyc_retail_school)

map_nyc_retail_school

In [ ]:
map_nyc_retail_school.save("../fig/part_2_school_vs_retailer.html")

In [ ]:
df = year_score_boroughs_avg.transpose().reset_index().rename(columns={"index": "Borough"});
df

### COMMENTS
As usual, top-right button on the map offers you more options to review. By default, you are viewing the ratio
between retailers and high school in each borough, and the mean distance from each high school to its nearest five
retailers, shown by the radius of each circle.
Interestingly, inverse relation is told by the geo-information. Roughly scanning the map, you would find that high
schools on Staten Island shares a large mean distance to its nearest retailers, while the retailer/school ratio is
highest among all the boroughs; you can also check the counter-intuitive phenomenon in Manhattan and Bronx, where the
 mean distances are kept short with low retailer/school ratios.
We can dive into the phenomenon from another perspective. Please switch the map to markers of high schools and
heatmap for retailers by clicking top-right button on the map and ticking these two layers. On Staten Island, things
could attribute to the uneven and unmatched distribution between school and retailers. Most of the retailers, seen from
heatmap, are located in the north, while some high schools locate themselves away from north, in the middle or even
south of the island. As for Manhattan, though with a lower retail/school ratio, the distribution patterns for them
are geographically similar, resulting in a much smaller mean distance from schools to retailers.
Both mean distance to retailers and retail/school ratio cannot be solid criteria for school scoring, since the
calculation does not involve the relationship between schools and legal tobacco retailers. However, out of reasonable
 suspect, we will go deeper into the problem in the next section using machine learning model via random forest
 algorithm.

In [ ]:
# for b in boroughs:
#     sub = df_school_directory[df_school_directory['Borough']==b]
#     plt.figure()
#     plt.title(b)
#     sns.histplot(sub['mean distance'])

row_num = 3;
col_num = 2;

fig, axes = plt.subplots(row_num, col_num, figsize=(10, 6), sharey=True, constrained_layout=True, dpi=300)
fig.suptitle('Mean distance from school to its closet five retailers', fontsize=16)

for (idx, b) in enumerate(boroughs_name):
    sub = df_school_directory[df_school_directory['Borough']==b]

    ax = axes[idx//col_num, idx%col_num];
    sns.histplot(ax=ax, data=sub['mean distance']);
    ax.set_title(b, loc="left");

axes[row_num-1, col_num-1].remove();
fig.savefig("../fig/part_2_mean_dist_bar.svg")

In [ ]:
row_num = 1;
col_num = 2;
fig, axes = plt.subplots(row_num, col_num, figsize=(10, 6), constrained_layout=True, dpi=300)

mean_dist=[]
for (idx, b) in enumerate(boroughs_name):
    sub = df_school_directory[df_school_directory['Borough']==b];
    mean_dist.append(sub['mean distance'].values);

    ax = axes[1];
    sns.kdeplot(sub['mean distance'].values, ax=ax, shade=None, label=b, cut=0)
    ax.legend();

### box plot
ax = axes[0];
ax.boxplot(mean_dist, labels=boroughs_name);
ax.set_ylabel("mean distance");
ax.set_title("box plot");

### kde plot
ax = axes[1];
# sns.kdeplot(mean_dist, ax=ax, shade=None)
ax.set_xlabel("mean distance");
ax.set_title("kde plot");
fig.savefig("../fig/part_2_mean_dist_box_kde.svg")

# Machine Learning

### 1. Task discription  
Idealy we shall plot hotzones of early smoking, in terms of any individul student's own risk behaviors and the environment based on his school (the school's supportive score and the distance to retailers), since youth smoking is an issue calling on efforts from multiple parties. The cautionary results would suggest special measures for the hotzones' authorities, such as goverment's education branches, tobacco braches and each community's own administrators to regulate the smoking behaviors among teenagers. In a more broad way, the results would be useful for any city management plan in the goal of youth tobbaco control, and furthermore the city's sustainability.

However, given the limited dataset and resources we can get, we are not able to distinguish the risk behavior levels down to each school, since the smallest geographical unit of YRBSS data is a whole district. Thus this project focuses on anylyzing how all kinds of individual behaviors and social environmental factors influence the smoking acquisition in a machine learning way, without geo-mapping.

All the following procedures will show what and how the dataset for training was constructed according to previous analysis, so that we can combine risk behaviors, school scoring and school-to-retailer distances to predict whether a student's would start smoking very early.

### 2. Preprocessing
#### 2.1 roughly filter YRBSS dataset
- The chosen target is the *age of start smoking* `q31` in 2019, so we only choose rows with **valid target `q31` value** of 2019
- check the answers from YRBSS, and exclude question attributes with **response rate** less than 70%


#### 2.2 encoding the features
- We stratify the target value to 3 categories: non-smokers (with q31 = 1.0, those who don't smoke), youger started smokers(q31 = 2.0, 3.0 or 4.0, those who first smoke before 13 years old), and older started (q31 = 5.0, 6.0 or 7.0, those who first smoke after 13 years old)
- `sitecode` is a nominal feature, to feed this feature into the model we have to transform it into numbers. Integers 0 to 4 respectively corresponds to `['NYG' 'NYH' 'NYI' 'NYJ' 'NYK']`


In [ ]:
all_questions = [q for q in df_boroughs5.columns if 'q' in q] 

# find the questions included in 2019's survey
attr19_null = []
for question in all_questions:
    if pd.notnull(df_boroughs5[df_boroughs5['year']==2019][question]).sum() < 0.7*len(df_boroughs5[df_boroughs5['year']==2019]):
        attr19_null.append(question)   
    print(question,y, pd.notnull(df_boroughs5[df_boroughs5['year']==2019][question]).sum())

In [ ]:
### store the dataset for machine learnning as `df`
df = df_boroughs5.loc[df_boroughs5[df_boroughs5['year']==2019]['q31'].dropna().index].drop(columns=attr19_null)
df['q31'].replace([2.0,3.0,4.0],[2.0,2.0,2.0],inplace=True)
df['q31'].replace([5.0,6.0,7.0],[3.0,3.0,3.0],inplace=True)

#### 2.3 impute nan values
- There are still lots of missing values in `df`, because the students are not always keen about filling in the survey, so we need to impute these values to make them as real as possible.
- The YRBSS data points sampled from the same `PSU` and `stratum` in the same borough are likely to be from the same class of the same school [ref](), so we can impute the **demographic features** with other valid values based on these. `gender` is imputed from the mode of the same (`PSU`,`stratum`) group, because we already know that male and female are roughly the same amount, so if a subgroup is one-gender dominated, there would be another subgroup is another-gender dominated, and eventually they counter off. It's not so good to guese `race` from mode, though, otherwise we make minority groups more minority, generating biased model on races. All statistical indicators are meaningless on race, because the numbers are manually assigned, and in fact we can't put a distance, mean value, or bigger/smaller on a race value. As a result, we randomly generate a race number. `age` is different from race, that we can actually compare ages, and therefore we simply use round up mean values to fill in.
- The **question answers** are hard to impute, so we assign a new value 0.0, as a new category, to the missing ones. This will sure influence the model, because they in fact would definitely have different answers but now they are treated as the same. We will see the results then.

In [ ]:
all_questions = [q for q in df.columns if 'q' in q] 
df[all_questions] = df[all_questions].fillna(0.0)

In [ ]:
for b in boroughs_code:
    
    sub = df[df['sitecode']==b]
    sample_group = sub.groupby(['PSU','stratum']).count().index
    
    for g in sample_group:
        sub_group = sub[(sub.PSU==g[0]) & (sub.stratum==g[1])]

        # fill gender with mode
        sub_group['sex'].fillna(
            stats.mode(sub_group.sex)[0][0],
            inplace=True
        )
        
        df.loc[sub_group.age.index,'sex'] = sub_group.sex

        sub_group['race4'].fillna(
            np.random.randint(1,6),
            inplace=True
        ) 
        df.loc[sub_group.race4.index,'race4'] = sub_group.race4
        
        # fill age with mean
        sub_group['age'].fillna(
            np.ceil(np.mean(sub_group.age.dropna())),
            inplace=True
        )
        df.loc[sub_group.race4.index,'age'] = sub_group.age


### 3. Impute external attributes from school analysis
We want to make use of the following features from Part2 School Analysis:  
- supportive environment score
- retailer to school ratio
- mean shortest distance to retailers

in which each school has a specific score and distance calculated, and a borough has a retailer to school ratio. Thus, the score and distance are uniformly sampled from each school, since the probability of one student coming from one school is equal. Students in one borough would have the same retailer to school ratio of that borough.

After this step, the dataset for training contains 8595 records.


In [ ]:
# impute 2019 scores by boroughs:
df_score = df_high_schools[df_high_schools['year']==2019][['DBN','boro','Borough','element_score']].dropna()

for i,b in enumerate(boroughs_name):
    df_sub = df[df['sitecode']==boroughs_code[i]]
    
    score_sub = df_score[df_score['Borough']==b].element_score.values
    density = retailer_density.loc[b].values
    distance_sub = df_school_directory[df_school_directory['Borough']==b]['mean distance'].values
    
    df.loc[df_sub.index,'score'] = np.random.choice(score_sub, size = len(df_sub))
    df.loc[df_sub.index,'distance'] = np.random.choice(distance_sub, size = len(df_sub))
    df.loc[df_sub.index,'density'] = np.array([density]*len(df_sub))
df

### 4. Build a decision tree model
#### 4.1 basic model setting 
- Training/test set split: 70% training set, 30% test set
- Model chosen: *Random Forest*  
The attributes are mostly categorical answers from a survey, so tree-based models would be suitable. The complexity would not be so high to overfit, and it is easy to tell whether one attribute plays an important role in classification. The advantage of Random Forest is that it offsets the data imbalance by bootstrap, which generates a new dataset sampled from the original one for each classification tree in the forest. We have seen that this dataset is not strictly balanced -- there are fewer records of Staten Island, and that's when random forest is more suitable.

#### 4.2 further attributes filter, to avoid data leakage
- The samples come from only high school students, so `age` is not randomly included -- as in Part1's demographic analysis, students are usually 14-17 years old. Since the target is also about age, and a student who is 15 should not choose that he starts smoking at 16 or 17, the `age` attribute is inherently correlated with the target, and as a result we should drop `age`.
- We already know that `age` and `grade` is highly correlated in Part1 YRBSS demographic analysis, so we don't include `grade` column to avoid data leakage. 
- Similarly, `q30`, whether smoked, and `q31`, initiation of smoking, have a causal relationship: if a non smoker is being honest, he should show consistency with his answers for `q30` and `q31`, and therefore we exclude `q30` as well. 
- Other features were then checked again on their number of missing values. The features still with small amount of missing values are kept with the missing values droped, and features with quite a few missing values are excluded completely. 


In [ ]:
le = preprocessing.LabelEncoder()
df['sitecode'] = le.fit_transform(df["sitecode"])

print('0 - 4 corresponds to', le.inverse_transform(range(5)))

In [ ]:
for c in df.columns:
    if  pd.isnull(df[c]).sum()>0:
        print(c,'number of nan', pd.isnull(df[c]).sum())

#### 4.3 train on the dataset of all students

In [ ]:
x = df[['sitecode', 'sex', 'race4', 'qnobese', 'qnowt', 'q66', 'q65', 'sexid',
       'sexid2', 'q12', 'q14', 'q15', 'q16', 'q17',
       'q20', 'q21', 'q22', 'q23', 'q24', 'q25', 'q26', 'q28', 'q29',
       'q31','q32', 'q33', 'q34', 'q35', 'q36', 'q38', 'q40', 'q41', 'q42',
       'q46', 'q47', 'q48', 'q50', 'q52', 'q53', 'q56', 'q58', 'q59', 'q60',
       'q61', 'q62', 'q69', 'q70', 'q75', 'q78', 'q79', 'q80', 'q81', 'q84',
       'q85', 'q86', 'q87', 'q88','score', 'distance', 'density']].dropna(subset=['race4','sexid','sexid2'])

y = x['q31']
x = x.drop(columns=['q31'])

### generate training/test set
### 70/30 train/set division, as requested
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

### train the random forest classifier
### this classifier uses default hyperparameters from sklearn
clf_4 = RandomForestClassifier(random_state=42)
clf_4.fit(x_train, y_train)

pred_y_train = clf_4.predict(x_train)

print('-------------------training set------------------')
mat = confusion_matrix(y_train, pred_y_train, labels=[1,2,3])
print("accuracy",'{:.2%}'.format(accuracy_score(y_train, pred_y_train)))
print('precision', mat[1, 1]/(mat[0, 1]+mat[1, 1]))
print('recall', mat[1, 1]/(mat[1, 1]+mat[1, 0]))
print(mat)



# print('-------------------test set performance------------------')
pred_y_test = clf_4.predict(x_test)
# print('confusion matrix:')
mat_all = confusion_matrix(y_test, pred_y_test, labels=[1, 2, 3], normalize='true')
# print("accuracy",accuracy_score(y_test, pred_y_test))
# print('precision', mat[1, 1]/(mat[0, 1]+mat[1, 1]))
# print('recall', mat[1, 1]/(mat[1, 1]+mat[1, 0]))
#print(mat_without)

#### 4.4 train on the dataset with only smoking students


In [ ]:
x = df[df['q31']>1][['sitecode', 'sex', 'race4', 'qnobese', 'qnowt', 'q66', 'q65', 'sexid',
       'sexid2', 'q12', 'q14', 'q15', 'q16', 'q17',
       'q20', 'q21', 'q22', 'q23', 'q24', 'q25', 'q26', 'q28', 'q29',
       'q31', 'q32', 'q33', 'q34', 'q35', 'q36', 'q38', 'q40', 'q41', 'q42',
       'q46', 'q47', 'q48', 'q50', 'q52', 'q53', 'q56', 'q58', 'q59', 'q60',
       'q61', 'q62', 'q69', 'q70', 'q75', 'q78', 'q79', 'q80', 'q81', 'q84',
       'q85', 'q86', 'q87', 'q88', 'score', 'distance', 'density']].dropna(subset=['race4','sexid','sexid2'])

y = x['q31']
x = x.drop(columns=['q31'])

### generate training/test set
### 70/30 train/set division, as requested
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

### train the random forest classifier
### this classifier uses default hyperparameters from sklearn
clf_4 = RandomForestClassifier(random_state=42)
clf_4.fit(x_train, y_train)

pred_y_train = clf_4.predict(x_train)

print('-------------------training set------------------')
mat = confusion_matrix(y_train, pred_y_train, labels=[1,2,3])
print("accuracy",'{:.2%}'.format(accuracy_score(y_train, pred_y_train)))
print('precision', mat[1, 1]/(mat[0, 1]+mat[1, 1]))
print('recall', mat[1, 1]/(mat[1, 1]+mat[1, 0]))
print(mat)



# print('-------------------test set performance------------------')
pred_y_test = clf_4.predict(x_test)
# print('confusion matrix:')
mat_smokers = confusion_matrix(y_test, pred_y_test, labels=[2, 3], normalize='true')
# print("accuracy",accuracy_score(y_test, pred_y_test))
# print('precision', mat[1, 1]/(mat[0, 1]+mat[1, 1]))
# print('recall', mat[1, 1]/(mat[1, 1]+mat[1, 0]))
# print(mat)

#### - Fig confusion matrix of models

In [ ]:
fig, axs = plt.subplots(1,2, figsize = ([14,6]))
fig.suptitle('Confusion matrix of the two models')

plt.subplots_adjust(wspace=0.3)

cf_all = ConfusionMatrixDisplay(confusion_matrix=mat_all, display_labels=['no smoking', 'start before 13', 'start after 13'])
cf_all.plot(ax=axs[0], cmap='crest')
axs[0].set_title("Confusion matrix of all students")

cf_smokers = ConfusionMatrixDisplay(confusion_matrix=mat_smokers, display_labels=['start before 13', 'start after 13'])
cf_smokers.plot(ax=axs[1],cmap='crest')
axs[1].set_title("Confusion matrix of smokers")

plt.savefig('../fig/part_3_cm.svg', bbox_inches='tight')

#### Comment
- The model on all students doesn't work well enough as expected. It shows an extreme bias on "no smoking", skewing the prediction very much. More than 60% of smokers who started smoking before 13 and about 75% of smokers who started after 13 were classified as non-smoking. Possible reasons of this lie in the sizes of these two groups. Compared to over 7000 records of non-smokers, the latter two only constitute about 1000 records. This lead to the model to be underfitting, and unable to learn the smokers' characteristics, compared to the non-smokers. The issue would be relieved if given a larger dataset.
- The model trained on only smokers are much less biased compared to the previous model. Both class reaches an accuracy of 70%. Considering the limited size, we can only claim the model is performing well within the existed dataset. The generalization performance is still in doubt.
- Some conclusions, or perhaps assumptions can still be drawn from the comparisons, regardless of the unsatisfying results. Non-smoking high school students would always be the majority, whether they would start smoking is not always predictable, but once they pick up smoking, there might be some pattern in all kinds of behaviors, where adults and administrations can get in the way, and imporse some positive effects.


#### 5. more insights on smokers

In [ ]:
importances = clf_4.feature_importances_;
std = np.std([tree.feature_importances_ for tree in clf_4.estimators_], axis=0)

forest_importances = pd.Series(importances, index= x.columns)
forest_importances = forest_importances.sort_values(ascending=False)

#### - Fig importance

In [ ]:
color = "#93B59B";
figsize = (10, 6);
x_tick_rot = 45;
### bar plot with stddev
fig, ax = plt.subplots(1, 1, figsize=figsize)
forest_importances[:20].plot.bar(yerr=std[:20], ax=ax, color=color, rot=x_tick_rot)
ax.set_title("Feature importances using MDI", fontsize=14)
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()
plt.savefig('../fig/part_3_importances.svg', bbox_inches='tight')

#### Comment
- Seeing from the top 20 most important features of the smoker model, `q40`, when started drinking alcohol, tops at first. This suggests that starting to smoke and starting to drink are not seperate events -- once the guard to one is down, the other might soon collapse. Similarly, `q46`, also in the top 5, describes the initiation age of marijuana usage. These three features combined informed us that regulations should not be isolated, and a sustainable city shall maintain a good balance in all. 
- `q59` describes the age of first sexual intercourse, which in combination with the tobacco, drug and alcohol complete a teenage group profile who are highly curious, impulsive and reckless, and eager to feel like adults by making not cool decisions. 
- `distance` and `score` are listed the 2nd and the 3rd, meaning that our assumption that the social environments influence smoking initiation is also proved to some extent. How this two feature differ the smoking initiation age is demonstrated in the following chart.
- `density`, on the other hand, is not as influential as the other two envrionmental factors. The impurity lessened by it alone is only half that of the other two. It seems it does not matter that much the average number of retailers against schools, but the specific neighborhood condition of each individual school contained in `distance`.
- Demographic features, such as `race` and `sitecode` are not very important in this model, even though we found different distributions of all years of 5 boroughs in YRBSS's analysis.


#### - Fig top different features of smokers

In [ ]:
top5 = ['q40','distance','score','q46','q59']
questions_top5 = ['age of starting drinking alcohol','distance to retailers', 
                  "school's supportive score", "age of starting using marijuana",
                  "age of first sextual intercourse"]
fig, axs = plt.subplots(5,2, figsize = ([12,18]), sharey=True)

fig.subplots_adjust(wspace=0.2, hspace=0.4)
i = 0

for q in top5:
    x = [axs[i,0],axs[i,1]]    
    x[0].set_xlabel(questions_top5[i])
    x[1].set_xlabel(questions_top5[i])
    if len(x_test[q].unique())>10:
        sns.histplot(x_test[pred_y_test==2][q], ax=x[0], color=color_list[0], bins=8)
        sns.histplot(x_test[pred_y_test==3][q], ax=x[1], color=color_list[0], bins=8)
    else:  
        x_test[pred_y_test==2][q].value_counts().sort_index().to_frame().plot(kind= 'bar', ax=x[0], rot=0, color=color_list[0], legend=False)
        x_test[pred_y_test==3][q].value_counts().sort_index().to_frame().plot(kind= 'bar', ax=x[1], rot=0, color=color_list[0], legend=False)

    i = i+1
    

axs[0,0].set_title('Group of starting smoking before 13')
axs[0,1].set_title('Group of starting smoking after 13')
plt.savefig('../fig/part_3_feature_diff.svg', bbox_inches='tight')
#plt.suptitle('Difference within smokers')

#### Comment
- The difference within smokers who start smoking at different ages show different patterns in all the features we explored. The age of smoking initiation seems to positively related with all these 5 factors: age of starting drinking alcohol, distance to retailers, school's supportive score, age of starting using marijuana, age of first sextual intercourse. Younger age of catching up alcohol, drug and sex often predicts younger age of smoking, too. Same with the school factors, if a school is located further from tobacco retailers, and receives a higher supportive score, the students attending this school are likely to start smoking later in life. 
- The lesson we can learn from this result complies with our common sense, but powerful: Keep your kid away from any of this: alcohol and drug, will help them stay away from tobacco, too. Sexual education, school's societal environmental building-up and a clean, peaceful neighborhood would support their healthy behaviors, and any administrator and stakeholder in the related departments should bare this in mind, to make their city more upright and sustainable for the younger people.

# Visualization
visualization in the project varies from static plots to interactive plots, from statistical plots to plain figures. We will discuss the choice of visualization in this section.

### Geo-plots
All geo-plots in the project in implemented with *folium* library. A typical geo-plot in the project should have layers whose visibility is customizable to readers by clicking the embedded layer button on the map. 

Choropleth plot appears most frequent in the layers, as one of the most common geo-data structure we were facing is the value distribution with respect to the five boroughs in NYC. Choropleth plot can precisely describe the geographical distribution using distinguishable colors. 

Markers are used only once for marking high school locations on the map. They are effective when comes to demonstrating exact locations of a dataset in a small scale on the map. However, things become messy when you try to mark too many locations on the map due to their memory consuming nature. Our machine died once for running more than 4,000 retailers markers on the map. That is also the reason we limit its usage.

Heatmap provides readers with straight and decent experience, which is the perfect choice for identifying where something occurs, and demonstrate areas of high and low density. We used heatmap for describing the retailer distribution and density. We found that heatmap shows its excellence also at the time when plotting the exact location is not important, or not possible: it does a good job leaving readers with quick impressions.

Combined with a time slider bar, choropleth in time series offers a power tool for visualization over time. Dataset with data from multiple years are shown with this type of visualization mostly. Through dragging the top bar, readers can check the differences across the timescale.

### Statistical Plots
Statistical plots are implemented in various ways. *Bokeh* library is applied for making interactive plots in particular.

The stacked bar plot, with fixed vertical length and flexible partial length for each element representing the accounted portion, makes itself much repetitive appearance in Part I. This type of plot offers an easy way for readers to compare among data from different items, say boroughs, while sometimes people may think pie chart is a little clumsy when comes to compare across.

Classic bar plot and line chart are always solid. Box plot gives a visual summary of the data , through which readers can quickly identify mean values, the dispersion of the data set, and signs of skewness. It is useful when we are talking about data distributions.

# Genre to choose
Built on the *Magazine* and *Annotated Graph* genres introduced in Segal and Heer paper, we applied several methods on visual presentation and narrative presentation to better help us tell an attractive and logical story.

### Visual Presentation
We believe that good visual presentation could facilitate readers understanding our intention hiding in the context. For instance, heavily relying on geo-plot to demonstrate information with respect to boroughs, the project introduces many interactive geo-plots plotted via *folium* library. In these plots, readers are able to interact by changing map tiles and data layers to combine the information they prefer. Other examples can also be found in plots from *Bokeh* library.

We also value the power of design. we would like to present our project on a simple, bold, powerful or even little aggressive website. Starting from an open-source template, we made decent modifications to achieve the desired visual effect as presented. With a you-can-not-miss-it title, readers could be attracted by the article even at a quick glance.

### Narrative Presentation
Separated into three parts, the project attempts to keep the information taken by readers in a fair amount on every reading. Each part on an individual but connected topic contains consistent and not too much information. Provided with detailed outline, readers can customize their reading experiences. 

# Discussion
### What Is Good
Franking speaking, we are satisfied with the topic of the project. Having had a friend who addicted himself to cigarette, we are happy to have the chance to dig into a problem that we actually care about. 

About the project contents, we do believe that it contains a large volume of work with decent analysis on three individual topics related to the project. In total we presented more than 30 plots in our project, as well as large amount of explanations. We do appreciate our time spent on the project by generating better analysis and reading experiences.

Diving into the data is always the toughest part, and we have successfully cracked the mystery inside with plenty of labor understanding them. Some data came in the form of questionnaire, where we had to pay extra work finding the explanation to each answer. To make things worse, data from different years came with different sets of answer, for which we had to do a large amount of data cleaning and merging.

Given large amount of visualization, we did a nice job delivering them in a various and interactive way. Tools such as *folium* and *Bokeh* libraries for interactive plots are applied for better visual effect. We made a great effort on creating interactive plots, especially for the geo-plots. We believe that we have achieved
our goal on visualizations at present.

Picking a right template for website producing also contributes to the overall visual effect of the project thus boost reading experience. Abundant explanatory texts also ensure that readers can reach our insight in a fast pace.

### What Is To Improve
- When working with data one should always be skeptical how real one dataset is representative of the whole population, and our dataset would not totally avoid bias, either. The dataset could be worked with more advanced statistical techniques to mitigate the bias and reach a more true reflection of everyone.
- To combine multiple datasets we did not use genuine data but simulated with possible values, this could possibly influence the machine learning model, and indicate false interrelationships within features.


# References
E. Segel and J. Heer, "Narrative Visualization: Telling Stories with Data," in IEEE Transactions on Visualization and Computer Graphics, vol. 16, no. 6, pp. 1139-1148, Nov.-Dec. 2010, doi: 10.1109/TVCG.2010.179.

[YRBSS main page](https://www.cdc.gov/healthyyouth/data/yrbs/index.htm)

[NYC school survey homepage](https://www.schools.nyc.gov/about-us/reports/school-quality/nyc-school-survey)

# Contribution
Sixian Zuo(s210365)
- Data processing for part I, II and III
- Data visualization for part I, II and III
- Model Training

Kewei Du(s220059)
- Data processing for part II
- Data visualization for part I and II
- Website Producing